# Quickstart: Vector search

Azure AI Search supports vector indexing and queries, useful for similarity search and information retrieval over multimodal content.

This is the source code for the article: [Quickstart: Vector search (Python)](https://learn.microsoft.com/azure/search/search-get-started-vector).


## Install packages and set variables

In [89]:
# Load environment variables from .env file
# Rename the samples.env to .env and fill in the values
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os

load_dotenv()

AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")

## Create an index

In [136]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    SimpleField,
    ComplexField,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    ExhaustiveKnnAlgorithmConfiguration
)

search_credential = AzureKeyCredential(AZURE_SEARCH_API_KEY)

index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=search_credential
)

fields = [
    SearchableField(name="project_month", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
    SearchableField(name="project_id", type=SearchFieldDataType.String, key=True,filterable=True, sortable=True, facetable=True),
    SearchableField(name="project_title", type=SearchFieldDataType.String),
    SearchableField(name="requirements", type=SearchFieldDataType.String),
    SearchableField(name="division", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchableField(name="registrant", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchableField(name="current_month_status", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SimpleField(name="headquarters", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="department", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="team", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="project_category", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="remarks", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="priority", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SimpleField(name="target_system", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
    SimpleField(name="registration_datetime", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
    SimpleField(name="effort_hours", type=SearchFieldDataType.Int32, filterable=True, sortable=True, facetable=True),
    SimpleField(name="unit_system", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
    SimpleField(name="previous_month_status", type=SearchFieldDataType.String, filterable=True, sortable=True)
]

vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw-vector-config-1", kind="hnsw"),
            ExhaustiveKnnAlgorithmConfiguration(name="my-eknn-vector-config", kind="exhaustiveKnn")
        ],
        profiles=[
            VectorSearchProfile(name="my-vector-profile", algorithm_configuration_name="my-hnsw-vector-config-1")
        ]
    )

semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
           title_field=SemanticField(field_name="project_id"), 
           content_fields=[SemanticField(field_name="project_title"),SemanticField(field_name="requirements")], 
           keywords_fields=[SemanticField(field_name="project_month"),SemanticField(field_name="current_month_status"),SemanticField(field_name="previous_month_status"),SemanticField(field_name="division")]
        )
    )

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

semantic_settings = SemanticSearch(configurations=[semantic_config])
scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['project_month', 'project_title', 'requirements', 'current_month_status']}]

# Create the search index with the semantic settings
index = SearchIndex(name=AZURE_SEARCH_INDEX, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 mvp-sample-vector-index created


## Create documents payload

In [137]:
# Create a documents payload
documents = [
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33428",
      "project_title": "수도권 도매채널 거래관계생성 BPM품의 호출조건 변경_대리점 매핑카운트 초과 시 품의작성 불가",
      "requirements": "1. 요약 - 대리점 매핑카운트 초과 시 품의작성 불가  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지) - RDS > 유통망 > 유통망관리 > 유통망정보관리  3. 변경 전 기능(현재 기능 및 문제점 기술) - 신규관계설정이 아래 조건 중 하나 이상 만족 시 BPM 관계설정품의 호출 (1) 수도권 도매영업부소속 대리점과 맺은 관계설정이 판매점 권역에 설정된 N건 초과 시 호출 (2) 해당 판매점 권역에 책임상태가 아닌 미책임 대리점이 관계설정 시 호출  4. 변경 후 기능(개선 될 내용 기술)  - 신규생성하는 거래관계가 기 매핑된 N값 카운트 초과 시 불가 팝업  . 매핑 카운트는 '품의 진행중', '거래상태 = 거래중'을 포함함  - 상세정보>대리점 매핑 가능건수도 동일 룰로 반영  - 대리점 매핑 카운트에서 제외되는 대리점 리스트 현행화(기존 기능에서 데이터만 현행화-첨부참조)",
      "registration_datetime": "2025-06-16 18:48",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선채널정책팀",
      "registrant": "안제세",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 8,
      "unit_system": "유통망",
      "previous_month_status": "신규",
      "remarks": "C_01",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-20567",
      "project_title": "W 초이스 후속 개발 (외부 연동/상품 상세)",
      "requirements": "25.4월 정기 과제로 배포된 {위버스 초이스 3종} 후속 개발  - 제휴사 정보 연동   - 상품 상세 스펙 후속 개발 (혜택 화면 조회 등)",
      "registration_datetime": "2025-04-08 10:09",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "요금혁신담당",
      "team": "무선요금혁신팀",
      "registrant": "최상균",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "보류",
      "remarks": "C_02",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-03945",
      "project_title": "KTS 업셀링을 위한 정산 고도화",
      "requirements": "1. 인터넷/TV 상품 접수 후 개통 시점의 상품 변경 오더 섹션 개발 -대상: 인터넷, TV -정의: 상품 접수 후, 개통 시점에 요금제 변경 오더 개발  2. 상품 변경 오더에 따른 정산 기능 개발  3. 기타 -최초 상품 접수 조직 > 동일 유지 -업셀링 조직 > 상품 변경 조직",
      "registration_datetime": "2025-01-20 15:45",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선유통혁신팀",
      "registrant": "김세아",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 15,
      "unit_system": "수수료",
      "previous_month_status": "보류",
      "remarks": "사업화 필요",
      "priority": "C_03"
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33426",
      "project_title": "수도권 도매채널 거래관계생성 BPM품의 호출조건 변경_특정 판매점 리스트 BPM품의 강제호출 적용의 건",
      "requirements": "1. 요약  - 판매점 조직관계 신규 > 미책임 대리점과 거래관계생성 시 BPM 호출  - 특정 접점코드의 경우 무조건 BPM 호출  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - RDS > 유통망 > 유통망관리 > 유통망정보관리  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 신규관계설정이 아래 조건 중 하나 이상 만족 시에만 BPM 관계설정품의 호출  (1) 수도권 도매영업부소속 대리점과 맺은 관계설정이 판매점 권역에 설정된 N건 초과 시 호출  (2) 해당 판매점 권역에 책임상태가 아닌 미책임 대리점이 관계설정 시 호출  4. 변경 후 기능(개선 될 내용 기술)  - 판매점 조직관계 신규>미책임 대리점과 거래관계생성 시 BPM 호출  *판매점 권역관리TAP에 설정된 권역별 N값 한도 동일 적용  *수도권 본부만 해당 동일 적용  - 특정 접점코드의 경우 무조건 BPM 호출  . 대리점 매핑 카운트 초과 시 품의 작성 불가 룰 동일 적용  . 특정 접점코드(자동적용): 해당 접점이 직전월~조회시점 내 실적이 없는 경우_RDS (조직별) 접점유형별실적 무선실적 조회 기준  . 특정 접점코드(수기적용): ITSM 데이터변경 요청",
      "registration_datetime": "2025-06-16 18:34",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선채널정책팀",
      "registrant": "안제세",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 12,
      "unit_system": "유통망",
      "previous_month_status": "신규",
      "remarks": "C_04",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-34209",
      "project_title": "RDS > 정책수수료 등록 > 부가단독 정산 로직 데이터 변경",
      "requirements": "RDS > 정책수수료 등록 > 실적등록 > 부가단독  ㅇ 부가 단독 가입 시 모바일 개통 실적 없는 접점도 정산 되도록 로직 변경 요청",
      "registration_datetime": "2025-06-19 14:41",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "송시욱",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "완료",
      "effort_hours": 12,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "C_05",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-38657",
      "project_title": "[디바이스단독]통신향 공단말의 삼디초 요금제 적용 과제",
      "requirements": "1. 요약  - 삼디초 요금제에 현재 운영 중인 1)통신향 자단말 2)기타기기 3) 통신향공기계 중 택1하여 할부금 할인 혜택 제공  - 공기계는 대리점 RDS 시점재고 보유 모델만 가능, 개통 조건이 아니며 고객 선택 사항임(할부 판매 -> 개통 OR 미개통)  - 공기계 할부와 공기계 실개통 명의자는 상이할 수 있음 (실개통시엔 자급제 단말처럼 개통 필요)   - 판매정책은 디단독 등록 화면통해 활용(모델 등록 / 할인금 등록 / 판매정책 생성등)  #개발 사항 PPT 참고(스펙회의 진행시 세부 내용 설명 예정)",
      "registration_datetime": "2025-07-10 09:22",
      "division": "Customer부문",
      "headquarters": "Device사업본부",
      "department": "Device마케팅담당",
      "team": "Device사업팀",
      "registrant": "원정섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "중기",
      "effort_hours": 15,
      "unit_system": "물류/채권",
      "previous_month_status": "신규",
      "remarks": "C_07",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-35859",
      "project_title": "하이오더/로봇 RDS > C타입(수기) 정산을 위한 개발 요청",
      "requirements": "ㅇ C부문 감사 조치 요구사항   1)획득-유지판촉비 계정 혼용 불가  2) 획득 세금계산서 정산 축소  3) 획득 정산 시 정책지 등에 정산 근거자료 필수 ㅇ 현재 하이오더와 로봇은 CONAN 전산을 통해 하이오더/로봇 모집수수료를 RDS 전송하는 형태로 운영중이며, BM외 수수료 로딩 권한이 없어 RDS 정산 개발을 통한 고객본부별 정산 기능 부여/관리 및 정책지 근거자료 등록, 로딩을 통한 투명한 정책 수수료 운영 목적 ㅇ 요청사항  - RDS > 수수료 > 정책지등록 >상품에 하이오더와 로봇 개발 필요(미래성장 > 하이오더 or 로봇 > 일반)  1차로 C타입(수동) 개발 후 완료 시 A,B타입(자동) 개발 예정   - 수수료코드 개발 필요(수수료코드세팅) : 하이오더 정책수수료, 로봇 정책수수료 등  . 현재 수수료 코드(하이오더 E106, 로봇 E104) 정책수수료 수용 가능한지 확인 후 필요 시 개발 진행 필요    DR-2025-35859  하이오더/로봇 RDS > C타입(수기) 정산을 위한 개발 요청   스펙회의   C타입 우선 개발 1. 우선순위에 따라 SR로 진행가능 2. SR로 개발시 LOB 미분리, 인터넷 하위에 포함 3. 데이터 확인 및 권한 필요   ->지급대상이 현재 정책수수료대상조회(원부)에 포함되는지   ->정책수수료대상조회 인터넷-기가아이즈 데이터 유통정산 팀에 요청 4. 추후에 A/B타입 개발시 마이그레이션 필요  추후 A/B 타입 개발시 필요 확인 사항 1. 사업화필요 2. LOB 분리 여부 3. 필요팩터 정리 4. 향후 사업화 규모 확인시 정책지 제공 필요 5. 사업화 준비 단계에서 추가 스펙회의 필요",
      "registration_datetime": "2025-06-26 17:40",
      "division": "Customer부문",
      "headquarters": "소상공인사업본부",
      "department": "소상공인사업담당",
      "team": "하이오더사업팀",
      "registrant": "김민규",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 15,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "C_08",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-37608",
      "project_title": "[대채기] RDS 정책수수료등록 하이오더 실적집계로직 개선개발",
      "requirements": "1. 요약  - 하이오더 실적집계 로직 개선 (현재 하이오더는 인터넷 하위에서 집계되고 있음)  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - RDS 정책수수료등록  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 하이오더 총단말댓수 (메뉴판+알림판+웨이팅단말 포함) 이 곧 실적인데, 현재는 1고객계약이 곧 1개의 실적으로 잡히고 있음.  4. 변경 후 기능(개선 될 내용 기술)  DR-2024-32861 개발 건 후속으로,  RDS 정책수수료대상조회 에서는 “카메라 대수”실적 로 정상적으로 잘 전송되는 것을 확인하였으나,  정작 B타입 수수료 등록시에는, 대리점별집계 엑셀다운로드로 확인하여 보니, “사업자당/고객당 실적”으로 잡히는 것으로 보입니다. Ex: 피앤에프 대리점삼거리점은 하이오더 실적 28건 (20건+8건) 을 인정받아야하나, 대리점별 집계로보면 2건으로 인정됩니다.   &lt;실적집계방식 개선제언/유통서비스팀> - 카메라댓수*실적건수 하는 방식으로 총단말댓수 팩터에 가중치 적용",
      "registration_datetime": "2025-07-04 16:22",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널기획팀",
      "registrant": "양혜주",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 23,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "C_09",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-29978",
      "project_title": "정리점 강제수납 제한 해제 개발요청2",
      "requirements": "1. 요약  -정리점 수납 가능토록 B2C 온라인 요금 기능 변경 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  -각종 요금수납화면 (즉시요금 포함) 3. 변경 전 기능(현재 기능 및 문제점 기술)  -요금수납 처리점이 '정리점'인 경우 수납처리 불가 4. 변경 후 기능(개선 될 내용 기술) ### B2C온라인요금 파트 1) 현재 수납처리점이 '정리점'인 경우, KOS에서 대상조직으로 조회조차 되지 않아 직권수납처리 아예 불가 2) 수납처리점을 전부 표출 (단, 현재와 같이 청산점은 제외) RDS와 연동하여,  .....RDS의 조직코드의 '상태'가 아닌 '수납 가능'여부를 회신받아 이후 진행여부를 확정 ### RDS파트 1) 대리점코드가 '정리점'으로 상태변경시(품의득결시), '수납가능여부'의 default값을 '수납불가'로 setting하고 사용자가 변경할 수 있도록(권한불요) - 제한구분 : 수납 / 제한사유 : 계약 / 제한상세사유: 사업자폐점 / 제한시작일자 : 배치처리일자 / 제한종료일자 : 9999-12-31 2) RDS에서 B2C온라인요금으로 회신시, (해당 조직코드의 계약코드의 '수납가능여부'가 Y/N) 에 따라 가능여부 회신",
      "registration_datetime": "2025-05-27 11:14",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "김준",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 7,
      "unit_system": "채권/유통망",
      "previous_month_status": "보류",
      "remarks": "오더 동시개발 필요",
      "priority": "C_10"
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-22014",
      "project_title": "소상공인 신규 제휴카드 연동개발",
      "requirements": "1. 요약  - 소상공인 신규제휴카드(KT 으랏차차 신한카드) 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - KOS/ RDS/ 신용카드   3. 신규 기능 상세 설명(추가될 내용 기술)  - MASS형 제휴카드(개인-개인사업자-법인사업자 등 전체)  고액카드 사용시 추가할인 제공",
      "registration_datetime": "2025-04-14 18:46",
      "division": "Customer부문",
      "headquarters": "소상공인사업본부",
      "department": "소상공인상품담당",
      "team": "소상공인상품제휴팀",
      "registrant": "배동훈",
      "project_category": "A - 필수 전략과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "사업부서 요청 보류",
      "remarks": "C_11",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-34412",
      "project_title": "eSIM 탑재 재생단말 수용을 위한 기등록 단말 원부 변경 조치 메뉴 개발 요청",
      "requirements": "1. 요약  - 반품/재생 단말 유통을 위해 기 등록된 단말 원부 정보 변경 필요  - 원부 정보 변경을 위해 KOS-RDS > 단말기 조회 메뉴 내 신규 메뉴 추가 요청  - 해당 메뉴를 통해 eSIM 재사용 대상 단말의 IMEI, EID 등 정보 업로드하여 단말 원부 등록 여부 조회  - 등록된 단말로 판단될 경우 기 등록원부의 EID 정보 변경, SM-DP Cancel Order 발송 처리 및 조치 결과 출력  - EID 정보 변경 및 Cancel Order 처리는 아이폰 리퍼 단말 처리 로직 활용 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - KOS RDS > 물류 > 단말기 조회 > (하단) 단말원부  - 단말원부 하위 메뉴로 '재생단말 원부변경' 신설 또는 '단말원부수정' 메뉴 내 하위메뉴 '재생단말 원부변경' 신설 3. 변경 전 기능(현재 기능 및 문제점 기술)  - eSIM 재사용 단말을 고려한 원부 관리 메뉴 없음 4. 변경 후 기능(개선 될 내용 기술)  - eSIM 재사용 단말을 위한 원부 관리 메뉴 신설",
      "registration_datetime": "2025-06-20 13:32",
      "division": "Customer부문",
      "headquarters": "Device사업본부",
      "department": "Device마케팅담당",
      "team": "단말서비스기술팀",
      "registrant": "이동식",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "중기",
      "effort_hours": 15,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "C_12",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33896",
      "project_title": "MVNO-RDS 보안취약점 개선 프로젝트 타시스템 연동테스트를 위해 개발",
      "requirements": "ㅇ 요청사항  [ BATCH ] 구분\t타입(inbound/outbound)\t대상 시스템\tendpoint/endpointInterface outbound\tKRDS\thttp://10.217.136.130:5000/SoapDynamicGateway outbound\tKCRM\thttp://10.217.136.127:8001/SoapGateway outbound\tORDR\thttp://10.217.136.217:5000/SoapGateway inbound\tMVNO(PORTAL)\tcom.kt.mvnords.wsclient.batch.OnDemandBatchPort  [ PORTAL ] outbound\tKRDS\thttp://10.217.136.130:5000/SoapDynamicGateway outbound\tMVNO(BATCH)\thttp://10.217.55.16:8080/services/OnDemandBatchWs outbound\tKCRM\thttp://10.217.136.127:8001/SoapGateway outbound\tKCDM\thttp://10.217.136.217:7100/SoapDynamicGateway outbound\tORDR\thttp://10.217.136.217:5000/SoapDynamicGateway  http://10.217.136.217:5000/SoapGateway inbound\tM-Platform\tcom.kt.mvnords.wsserver.MvnoIntmUpldInfoWs.MvnoIntmUpldInfoWSIF inbound\tKRDS(wowple)\tcom.kt.mvnords.wsserver.AprdiBizrAcruRetvWs.AprdiBizrAcruRetvWsIF  [ FILE 연동 ] - KOS-G/W : KAIT 사전승낙정보 7종 - M-PLATFORM : FDH 연동 1종",
      "registration_datetime": "2025-06-18 13:26",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "MVNO채널담당",
      "team": "MVNO제휴팀",
      "registrant": "김찬겸",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 7,
      "unit_system": "수수료/물류/실적/채권",
      "previous_month_status": "신규",
      "remarks": "C_13",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-32526",
      "project_title": "Watchdog 점검 지표에 신규 지표 추가 요청의 건(유통점 청구주소/납부자여부)",
      "requirements": "*Watchdog 점검 지표에 신규 지표 추가 요청의 건(유통점 청구주소/납부자여부)  ㅇ요청사항 -대상 : Watchdog Mobile/Internet/TV -추가항목: 유통점 청구주소 여부, 유통점납부자여부  (현재 SQMS 점검 항목) -위치: '가입' 카테고리에 배치 -시간 소요 시 Mobild->Internet->TV 순으로 반영",
      "registration_datetime": "2025-06-11 16:44",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "구도윤",
      "project_category": "A - 필수 전략과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 20,
      "unit_system": "실적",
      "previous_month_status": "이월",
      "remarks": "공수부족",
      "priority": "C_14"
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-34376",
      "project_title": "MVNO-RDS 유심인증정보 연동 제외 개발",
      "requirements": "ㅇ 배경   - K-RDS 에서 MVNO-RDS 로 유심 인증정보 (KI, OPC) 값을 연동받고 있음 (EAI 연동)  - MVNO-RDS 에서는 해당 값을 사용하지 않고 저장만 하고 있음  * 개인정보보호팀에서 사용하지 않으면 삭제하라는 권고 조치   ㅇ 현황 :   인터페이스아이디: NIFI_NRDS_NMRD_E_2009_00    [K-RDS] [MVNO-RDS]  테이블명 : IFS_LO_WLESS_DTL_MVNO -> LO_WLESS_INTM_ADTN_DTL    컬럼정보 : USIM_CUST_INFO_ECOD_VAL : USIM고객정보암호화값 (USIM KEY 에 대한 암호화 값)  USIM_GVORD_INFO_ECOD_VAL : USIM발주정보암호화값 (OPc에 대한 암호화 값) ㅇ 요청사항  - 사용하지 않는 위 2개 컬럼정보 EAI 연동 제외",
      "registration_datetime": "2025-06-20 10:36",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "MVNO채널담당",
      "team": "MVNO제휴팀",
      "registrant": "김찬겸",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "C_15",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-08407",
      "project_title": "KOS RDS 내 '대리점별 집계'화면 내 연계단말 처리 유통조직(대리점) 정보 추가요청",
      "requirements": "1. 요약  - '대리점별 집계' 내 연계단말 대리점코드, 대리점명 2개 컬럼 추가 요청  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - KOS RDS > 대리점별 집계 화면 3. 변경 전 기능(현재 기능 및 문제점 기술)  - '대리점별 집계'화면 내에 대리점정보 미표기됨 (2nd Device 정책 지급기준 (대리점 등) 확인을 위해 필요) 4. 변경 후 기능(개선 될 내용 기술)  - - '대리점별 집계'화면 내에 대리점정보 표기",
      "registration_datetime": "2025-02-12 17:05",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "수수료 공수 부족",
      "priority": "C_17"
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-20613",
      "project_title": "RDS 내 정책별집계 내 전체지급취소 환수여부 표기",
      "requirements": "1. 요약  -ㅇ개발SR등록요청   - 정책별집계 내 전체지급취소 환수여부 표기  . 결산마감 후 정책수정>전체지급취소 입력하여 전체환수 처리된 건에 대해서 전체지급취소 표기   2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - RDS 정책별 집계 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 4. 변경 후 기능(개선 될 내용 기술)  -ㅇ개발SR등록요청   - 정책별집계 내 전체지급취소 환수여부 표기  . 결산마감 후 정책수정>전체지급취소 입력하여 전체환수 처리된 건에 대해서 전체지급취소 표기    스펙회의  DR-2025-20613 RDS 내 정책별집계 내 전체지급취소 환수여부 표기  ㅇ 정책별집계>환수사유가 전체환수일 때   - 정산구분 컬럼에 전체환수로 변경 표기   - 과거데이터 마이그: 2018년~ (최대기간 확인필요)",
      "registration_datetime": "2025-04-08 10:46",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "중기",
      "effort_hours": 25,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "C_19",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-27337",
      "project_title": "RDS 시점재고 메뉴의 재고조정 버튼 숨김 처리",
      "requirements": "O 요청메뉴 : RDS > 시점재고 > 좌측 하단 재고조정 버튼 O 요청목적 : 현장 혼선 방지 O 개발요청 : 위 이슈가 발생하지 않도록 재고조정 버튼 숨김 처리",
      "registration_datetime": "2025-05-13 15:23",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "김태현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "물류",
      "previous_month_status": "이월",
      "remarks": "C_21",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-12778",
      "project_title": "마케팅부 지사 표기",
      "requirements": "ㅇ 요청메뉴 : RDS 전체 ㅇ 요청목적 : 휴먼에러 방지 및 업무효율 개선 ㅇ 요청상세 : 시점재고 > HUB별 경과일수 관리 팝업에서 마케팅부, 마케팅 1,2부로 표기되어 ㅇㅇ지사 마케팅부로 변경 요청  관련 마케팅부, 마케팅 1,2부 표기 건 일괄 변경 요청",
      "registration_datetime": "2025-03-05 11:58",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "손석민",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "물류",
      "previous_month_status": "이월",
      "remarks": "공수부족",
      "priority": "C_23"
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-09797",
      "project_title": "K-RDS 정책수수료 Grade 단가 환수 기능 보완 요청",
      "requirements": "1. 요약  정책단가 > 순위, 선착순, 달성횟수, 연속달성 단가 사용 시 Grade 환수 필요  - 최소건수 같이 체크 필요  - 순위 변동에 따른 재지급은 없고, Grade 구간 하락에 따른 환수만 있음  - 건별지급 / 일괄지급 모두 Grade 재계산 대상  - 정책단가 > Grade 단가와 동일한 룰로 환수사유 입력  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - KOS-RDS > 정책지 등록  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 정산 단가환수 시 세부기능 불가능   4. 변경 후 기능(개선 될 내용 기술)  아래 기능 가능  정책단가 > 순위, 선착순, 달성횟수, 연속달성 단가 사용 시 Grade 환수 필요  - 최소건수 같이 체크 필요  - 순위 변동에 따른 재지급은 없고, Grade 구간 하락에 따른 환수만 있음  - 건별지급 / 일괄지급 모두 Grade 재계산 대상  - 정책단가 > Grade 단가와 동일한 룰로 환수사유 입력",
      "registration_datetime": "2025-02-18 16:59",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 60,
      "unit_system": "수수료",
      "previous_month_status": "보류",
      "remarks": "수수료 공수 과다",
      "priority": "C_25"
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-20615",
      "project_title": "2G 가입자 5G 전환신규 처리 불가오류",
      "requirements": "가능하도록 개발요청,1. 요약  -ㅇ 개발SR등록 요청  - (기존) 2G가입자 -> 5G 변경 시, 현재 2G서비스종료로 해지만 가능해 010신규 개통 처리.   010신규 시 최초개통서비스계약ID 승계하여 당월 동일명의 해지 사유로 수수료 지급 대상 제외 중임.  - 위 경우, 2G-> 5G 이동한 고객이어서 수수료 대상건으로 로직 변경 필요 (2G 잔여가입자: 9,927건)  - (결정사항) 수수료 지급 기준, 010신규 or 전환신규인지 의사결정 필요 & 그에 따른 개발 변경 SR 등록 요청  : 010신규 처리 시, 당월해지여부 확인 시 기존 2G건 제외/ 전환신규 처리 시, 개발부서 논의 필요 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - RDS 정책별 수수료 집계 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 4. 변경 후 기능(개선 될 내용 기술)  -ㅇ 개발SR등록 요청  - (기존) 2G가입자 -> 5G 변경 시, 현재 2G서비스종료로 해지만 가능해 010신규 개통 처리.   010신규 시 최초개통서비스계약ID 승계하여 당월 동일명의 해지 사유로 수수료 지급 대상 제외 중임.  - 위 경우, 2G-> 5G 이동한 고객이어서 수수료 대상건으로 로직 변경 필요 (2G 잔여가입자: 9,927건)  - (결정사항) 수수료 지급 기준, 010신규 or 전환신규인지 의사결정 필요 & 그에 따른 개발 변경 SR 등록 요청  스펙회의  DR-2025-20615 2G 가입자 5G 전환신규 처리 불가오류, 가능하도록 개발요청 ㅇ 정책부서 의견 확인필요   - 5G 전환신규로 수집 필요 시 KOS-오더 개발필요  : 010신규 처리 시, 당월해지여부 확인 시 기존 2G건 제외/ 전환신규 처리 시, 개발부서 논의 필요",
      "registration_datetime": "2025-04-08 10:48",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "오더 개발필요",
      "remarks": "C_27",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-32048",
      "project_title": "지니TV 신규상품 출시 개발 요청 건",
      "requirements": "1. 요약  - 지니TV 신규 상품 출시 개발 건",
      "registration_datetime": "2025-06-10 10:27",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "요금혁신담당",
      "team": "유선요금혁신팀",
      "registrant": "김민화",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "이월",
      "previous_month_status": "C_28",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33134",
      "project_title": "따로살아도가족결합 원부 생성 로직 보완 요청",
      "requirements": "1. 요약  - 따로살아도가족결합 원부 생성 로직 보완 요청 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  -IT로직 개선 3. 변경 전 기능(현재 기능 및 문제점 기술)  -로직개선 전 원부 기준으로 정산한 5,6월 정책 대상 동일 케이스 현황 파악을 위해 자료 요청SR 등록 및 현황 공유 예정 4. 변경 후 기능(개선 될 내용 기술)  - 결합 약정갱신 OR 결합 기가전환으로 수집된 결합 내에 인터넷 신규건이 포함된 경우 & 반대로 신규에서 약갱 포함된 경우 비교",
      "registration_datetime": "2025-06-16 09:43",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선정책기획팀",
      "registrant": "김나영",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "완료",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "C_29",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-04798",
      "project_title": "KOS-RDS 와치독 기능 개발_IoT",
      "requirements": "요약  1. 현재 무선사업에서 RDS를 통해 활용중인 와치독 기능을   IoT대리점 뿐 만 아니라 KT영업대표들에게 확대 적용 하는 형태로 부분 개선 추진",
      "registration_datetime": "2025-01-22 16:27",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "IoT서비스1팀",
      "registrant": "김해건",
      "project_category": "A - 필수 전략과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "통합 고도화 진행 예정",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-06365",
      "project_title": "실적팩터 개선(부가서비스",
      "requirements": "당월해지후재가입),- 부가서비스",
      "registration_datetime": "당월해지후재가입 실적팩터 개선  : 개선사항 : 기간 시작일시~종료일시 자동세팅(결산월 기준)  예시)결산월 : 2025.02 -> 2025-02-01-00:00:00~2025-02-28 23:59:59",
      "division": "2025-02-04 13:28",
      "headquarters": "재무실",
      "department": "재무서비스센터",
      "team": "유통정산팀",
      "registrant": "경진혜",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 6,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-06412",
      "project_title": "요금제 그룹내 중복요금발생시 가중치 높은 항목 우선 지급",
      "requirements": "1. 요약  -전사 다운로드하여 오류내역 파악시 등록자명이 안보여 추가 확인하여야 함 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  -수수료>정책수수료통계>정책별실적세부내역 3. 변경 전 기능(현재 기능 및 문제점 기술)  -실적별실적집계내역 엑셀다운로드시 1행만 등록자명, 등록자사번 표시 4. 변경 후 기능(개선 될 내용 기술)  -실적별실적집계내역 엑셀다운로드시 모든행에 등록자명, 등록자사번 표시 요청",
      "registration_datetime": "2025-02-04 14:50",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "",
      "team": "유통정산팀",
      "registrant": "강은희",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-09413",
      "project_title": "[제휴부가]신규 CP 제휴 부가서비스 'AI 범죄차단 서비스' 개발 요청",
      "requirements": "1. 서비스 내용   - CP형 제휴부가서비스   - 서비스명 : AI범죄차단(가제)  - 월정액 : 5,500원 (미정)  - 서비스 내용: 딥페이크 텔레그램 URL 탐지 및 접속 차단, 딥보이스 의심 통화 내용 AI음성 분석, 피해 보상 지원  - 제휴사 : 헥토이노베이션  2. 개발 스펙  - 기 제휴 부가서비스와 동일 스펙 (KOS가입불가, 해지 가능)  (SHUB가입/해지) - 셀프케어 마이페이지 해지 가능(가입 당일 해지 가능) - 미성년자/선불/법인 가입불가 - MVNO 가입 허용 - 가입/해지 시 별도 SMS 발송 (SMS ID 추후 전달)  *MNO/MVNO 가입자 SMS ID 각각 적용, MVNO가입자는 파라미터값 (MVNO사업자명)적용  - 과금 6개월 미사용 무과금 - 정산은 K세틀 통해서 진행예정 (정산률 미정)  3. 기타 - 별도 배타 부가서비스 없음. - 단말-요금제 세대 불일치 등 관계없이 가입 가능 - 단말 제약 없음 - 출시 일정: 3Q내",
      "registration_datetime": "2025-02-17 16:59",
      "division": "Customer부문",
      "headquarters": "서비스Product본부",
      "department": "신규서비스담당",
      "team": "결제인증서비스팀",
      "registrant": "황보영",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-09657",
      "project_title": "부가서비스관리",
      "requirements": "추가 버튼 추가,부가서비스 관리 : 추가 버튼 필요 -> 등록 형식 부가서비스코드",
      "registration_datetime": "2025-02-18 14:20",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "",
      "team": "유통정산팀",
      "registrant": "경진혜",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-11795",
      "project_title": "TV/디바이스 매핑 로직 수정",
      "requirements": "1. 요약  - TV회선과 디바이스 단말 매핑시 1:1 매핑되어야 하나 1:2로 매핑되는 경우가 있음(디바이스 단말 이종인 경우) 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  -  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 변경 전 기능 : 디바이스 단말이 다른 모델로 N개 개통되는 경우 TV - 디바이스 단말 매핑이 1:N 으로 매핑되어 정산  - 변경 후 기능(개선 될 내용 기술)  . TV-디바이스 단말 매핑은 1:1 원칙으로 정산되도록 개선 필요  . TV 실적내 임대장비 팩터 미사용시에도 1:1 매핑 원칙 지켜지도록 개선 필요",
      "registration_datetime": "2025-02-27 10:09",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "",
      "team": "유통정산팀",
      "registrant": "김영자",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 15,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-14675",
      "project_title": "RDS '저소득층 지급취소' 조건 선택 체크박스 기능 비활성화 요청",
      "requirements": "1. 요약  - '저소득층지급취소' 조건 선택하는 체크박스 기능 비활성화 요청 (클릭시 체크 안됨) 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 대상 상품 : 인터넷(단품), 결합, 통합  - A타입(정책지등록) : RDS>수수료>정책지등록>지급취소조건탭> 지급취소기준  - B타입(정책수수료등록) : RDS>수수료>정책수수료등록> 지급취소등록탭> 지급취소기준추가  3. 변경 전 기능(현재 기능 및 문제점 기술)  -  4. 변경 후 기능(개선 될 내용 기술)  -",
      "registration_datetime": "2025-03-12 14:59",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "",
      "team": "유통정산팀",
      "registrant": "이수희",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-15160",
      "project_title": "타사고객 번호이동 후 RDS 리워드 사용내역 누락 개선",
      "requirements": "1. 요약  -타사고객 번호이동 후 RDS 리워드 사용내역 누락 개선  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  -N-RDS->리워드사용내역 3. 변경 전 기능(현재 기능 및 문제점 기술)  -리워드 사용 내역 내 선물하기 사용 내역 누락 4. 변경 후 기능(개선 될 내용 기술)  -리워드 사용 내역 내 선물하기 사용 내역 추출 가능",
      "registration_datetime": "2025-03-14 10:00",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "임세인",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "이월",
      "previous_month_status": "개발 진행 시 CRM하고 협의 필요",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-16148",
      "project_title": "[개발요청] 유선상품 사업용회선 해지를 위한 프로세스 개선",
      "requirements": "1. 요약  - 매장 패점 시, 유선사업용회선도 반납/해지 처리 되어야 하나 누락 발생   - 매장 폐점진행을 위해 RDS에서 폐점 품의 작성 선택 시, BPM을 통한 사업용회선 반납/해지 페이지 자동 팝업 발생토록 하여  사업용회선 먼저 반납/해지 처리 되도록 진행 ※ RDS 폐점 품의 내용 중, 유선상품 사업용회선 반납 여부 체크 메뉴 추가",
      "registration_datetime": "2025-03-19 11:22",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "사업지원담당",
      "team": "인터넷사업지원팀",
      "registrant": "김홍수",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 17,
      "unit_system": "유통망",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-18087",
      "project_title": "RDS 전자계약관리 상 불필요 기능 제거 요청",
      "requirements": "1. 요약  - ACE로부터 전자문서 수신 연동 제거, 전자문서 다운로드 기능 변경  2. 요청 기능 서비스의 메뉴 위치  - RDS > 유통망 > 전자계약관리  3. 요청사항  - 전자문서 보관을 RDS에서 ACE로 이관하는 것으로 결정되어 관련된 RDS 기능의 제거, 변경 반영 필요  - 전자문서 전송(ACE -> RDS) 연동 기능 제거 바람  - 전자문서 및 증빙서류 다운로드는 RDS -> ACE 연동으로 기능 변경 바람",
      "registration_datetime": "2025-03-27 10:33",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "AECS개발시 개발",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-18088",
      "project_title": "RDS 전자계약 보관 기능 이관 및 권한 확인 연동 기능 구현 요청",
      "requirements": "1. 요약  - 전자문서의 ACE시스템 보관 및 조회 권한 연동 기능 추가  2. 요청 기능 서비스의 메뉴 위치  - RDS > 유통망 > 전자계약관리  3. 요청사항  - 전자문서 보관을 RDS에서 ACE로 기능 이관 필요 (RDS 보관 기능 Decom)  - RDS에서 전자문서 조회 시 ACE와의 연동 기능 구현 바람  - RDS에서 대리점 조회 시 적용되는 소속조직 및 권한에 의한 제약을 ACE 보관 전자문서 조회 시에도 동일 적용하기 위한 조직 및 권한 확인 연동 기능 구현 바람",
      "registration_datetime": "2025-03-27 10:34",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "AECS개발시 개발",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-18089",
      "project_title": "RDS 전자계약관리 상 매장계약 기능 추가 개발 요청",
      "requirements": "1. 요약  - 매장 계약 관련 기능 추가  2. 요청 기능 서비스의 메뉴 위치  - RDS > 유통망 > 전자계약관리  3. 요청사항  - 전자계약 그룹화 단위를 계약점코드이나, 매장 계약에 대해서는 접점번호로 그룹핑 되도록 기준 추가 반영 필요  - 계약서(약정서) Naming Rule에 접점명, 접점번호 추가 바람  - 전자계약 조회 조건이 대리점코드 뿐이나, 유통망정보 조회와 동일하게 접점명, 접점번호로 조회할 수 있도록 기능 추가 바람  - 대리점 전자계약 시스템 연동에 추가될 접점명, 접점번호 반영 필요",
      "registration_datetime": "2025-03-27 10:34",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "AECS개발시 개발",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-20340",
      "project_title": "[요금제 수정] IoT 안심결제 요금제 스펙 수정 요청",
      "requirements": "1. 상세 요구사항  - 요청내용 : 지정된 Iot요금제에서 스마트폰 모델의 단말도 개통이 가능하고, 012번호로 음성통화시에 010번호로 변환하거나, 음성통화가 불가능하도록 부가서비스를 가입하는 형태로 개발 필요  - IoT요금제 : IoT안심결제 55 /77 / 110 / 165  - 단말모델명 : 2409BRN2CL (레드미 14c 스마트폰)  2. 요청 기능이 추가되어야 할 서비스 위치  - KOS에서 지정된 IoT요금제로 개통시에 스마트폰 단말로 개통하는 부분 허용이 가능하고,   부가서비스를 가입을 통해서 음성통화불가가 되어야 하며, 012번호로 발신이 되지 않고, 010번호로 발신이 되도록 변경이 필요  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 지정한 IoT요금제(Iot안심결제 55/77/110/165)에서는 스마트폰 모델로 개통이 불가하며, IoT모델(M2M)만 요금제 개통이 가능  - iot요금제와 iot단말 모델로 가입을 해도 발신통화시에 012 번호로 발신이 되고 있어서 WCDMA 약관상 이슈가 발생  4. 변경 후 기능(개선 될 내용 기술)  - 지정된(IoT안심결제 55/77/110/165)요금제에 스마트폰단말도 신규/기변 개통이 가능  - 지정된(IoT안심결제 55/77/110/165)요금제에서는 음성통화가 불가능하도록 자동지정 또는 부가서비스 가입이 가능  - 지정된(IoT안심결제 55/77/110/165)요금제에서 통화시에 012번호 표시가 아니라 010 번호로 발신표시가 가능",
      "registration_datetime": "2025-04-07 14:28",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "기업무선/IoT결제서비스팀",
      "registrant": "구대환",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "오더 개발시 진행",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-21087",
      "project_title": "MVNO 제휴 서비스에 대한 제휴사업자간 정산 프로세스 개선",
      "requirements": "독자전산 사업자의 제휴서비스 정산 누락 개선 및 정산내역 검증을 위한 프로세스 개선을 위해 자료 수집/집게/생성, 제휴사업자와 정산자료 연동 및 검증, 정산대상 자료 다운로드 등의 절차를 보완/추가하고자 함  ** 상세 요구사항은 첨부 파일(제휴서비스 정산 방안) 참조 요망",
      "registration_datetime": "2025-04-09 16:32",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "고객보호담당",
      "team": "고객보호팀",
      "registrant": "김규현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-21487",
      "project_title": "“단골고객” 이관 시 발송 문자 SRTT 적재되도록 요청",
      "requirements": "[요건] 단골고객 이관 시, - 동일 대리점 코드 내에서만 이관 가능 (다른 코드 대리점으로 이관할 경우 RDS에서 불가 팝업 노출) - 이관 당일 오전 10시 30분 해당 고객에게 안내 문자 발송 (사업부서 메시지 기획 예정) EX) 고객님의 전담 단골 매장이 A에서 B로 변경되었습니다. ~~~~ - 발송된 안내 문자는 해당 고객 SRTT에 적재 - 해당 고객에게 등록되어있던 약속 데이터도 함께 이관",
      "registration_datetime": "2025-04-11 10:30",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "김혜옥",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "유통망",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-21754",
      "project_title": "인터넷",
      "requirements": "TV 업셀링 정책 관련 자동화 팩터 개발,1. 요약  - 기가전환(IBA-08), 약갱업셀링정책(IBA-11) 수기(반자동화) 작업과다, 자동화 팩터 개발요청  2. 변경 후 기능(개선 될 내용 기술)  - 기가전환등 업셀링 관련하여 현재 반자동화로 운영중이나 자동화를 위한 팩터 개발 요청  - 기가역전환(기간내 다운셀링), 약정갱신과 동시판매등 팩터 개발 필요",
      "registration_datetime": "2025-04-14 09:46",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선케어정책팀",
      "registrant": "임명섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 20,
      "unit_system": "수수료",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-21871",
      "project_title": "KOS-RDS상에 업무처리조직 순서 변경 요청",
      "requirements": "○ 사용자 : 손강희 (10142624) ○ 메뉴 명 : KOS-RDS상에 업무처리조직 순서 변경 요청 ○ 문의 내용   가.현재 본인 기준 업무처리조직이 2가지로 세팅되어 있음  나.첫번째는 시너지영업부(546344), 두번째는 마케팅부(546343)으로 되어있음  다.상기 나에 대한 순서를 변경 요청 (첫번째는 마케팅부, 두번째는 시너지영업부)  마케팅부가 원소속이다 보니, 업무처리할 때 시너지영업부로 세팅되어서 계속 변경해야 함에 대한 불편함 존재  감사합니다",
      "registration_datetime": "2025-04-14 13:42",
      "division": "부산/경남광역본부",
      "headquarters": "부산/경남고객본부",
      "department": "창원지사",
      "team": "마케팅부",
      "registrant": "손강희",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "개발불가 사유: 업무처리조직은 조직별 매핑이므로 개인별 설정불가 조직별로도 표시순서를 설정할 수 있는 기능 미존재(필요 시 대면채널기획팀과 협의하여 구조 변경 필요)",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-24658",
      "project_title": "안심번호 SA대리점 위탁상품 적용 수수료 추가 개발(담당자:나혜정부장님)",
      "requirements": "1. 요약  - 안심번호 SA대리점 위탁상품 적용 개발 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 기본요금의 획득/유지수수료만 있음  - 고객에게 발생하는 접속료 매출에 대한 유지수수료 정책 추가 개발 요청 4. 변경 후 기능(개선 될 내용 기술)  - 접속료 매출의 10%를 유지수수료로 지급하는 정책 추가",
      "registration_datetime": "2025-04-28 16:02",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "커뮤니케이션서비스담당",
      "team": "기업지능망서비스팀",
      "registrant": "전미옥",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "개발 가능 여부 검토중",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-27510",
      "project_title": "RDS 조직(상품판매조직)실적팩터 보완 요청",
      "requirements": "- 조직(상품판매 조직) 실적팩터 보완요청  : 제외 대리점 -> 추가 버튼 클릭시 : 그룹구분_팩터 외 타 조건 선택 할수 없음 -> 대상대리점과 동일하게 그룹 선택 가능하게 보완",
      "registration_datetime": "2025-05-14 11:30",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "",
      "team": "유통정산팀",
      "registrant": "경진혜",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 6,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-27675",
      "project_title": "RDS 매장채권 파일 로딩시 부가세 항목 통제",
      "requirements": "1. 현황: RDS 매장채권 파일 로드시 통제 없이 입력값 그대로 로딩 2. 문제점: 부가세 가 청구액의 10% 범위 밖에도 파일값 그대로 로딩되어 과다/과소 청구 및 세무 리스크 존재  *실제 강남고객본부 12개 대리점 부가세 0.25억원 부가세 오청구 발생 (첨부파일)  예시: 청구금액 400만원, 부가세 440만원 입력  2. 개선요청 : 항목명이 원금인 경우 부가세금액이 청구액 의 10% 범위 벗어날 경우 로딩 실패 및 부가세 금액오류 팝업 알람",
      "registration_datetime": "2025-05-14 17:31",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "",
      "team": "Customer회계팀",
      "registrant": "김희영",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 8,
      "unit_system": "이월",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-28917",
      "project_title": "전화투표 서비스 수수료 지급 방식 변경",
      "requirements": "1. 요약  - 전화투표 수수료 지급 기준 변경  . 현재) 트레픽 기준 수수료 정산 -> 변경)수납 기준 수수료 정산  . 전화투표 청구항목 (LCLQ8ZJ, LCLQ2ZI, LCL0075)의 수납기준 정산  . 수수료 지급율 변경  ※ 유통서비스팀 권봉주 차장 협의",
      "registration_datetime": "2025-05-21 14:06",
      "division": "Customer부문",
      "headquarters": "소상공인사업본부",
      "department": "소상공인상품담당",
      "team": "소상공인통화상품팀",
      "registrant": "이국희",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 7,
      "unit_system": "수수료",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-30504",
      "project_title": "IoT 대용량 QoS요금제 6종 개발(LTE/5G)",
      "requirements": ". IoT 요금제 신규개발(6종) - 문제점:경쟁사(LGU)는 대용량 월데이터(20GB~500GB) 제공후 QoS로 전환(KT는 소용량3~30G로 열위) → 이에 경쟁사 수준 대용량 IoT QoS 요금제를 신규 개발  2. 유사개발은 23년 진행하였던(요청자 임종섭 차장) IoT QoS 3종과 스펙유사하나 차이점은 이번신규 요금제는 월 데이터가 대량이며 월데이터 소진후 QoS적용  상세 개발 신규 요금제는 첨부 파일에 상세 설명",
      "registration_datetime": "2025-05-29 09:53",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "기업무선/IoT관제서비스팀",
      "registrant": "임종섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "이월",
      "previous_month_status": "오더 개발시 진행",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-30733",
      "project_title": "[대채기] 비즈나루-RDS 수수료 연동 로직 개선 요청",
      "requirements": "1. 요약  - 비즈나루-RDS 수수료 연동 로직 개선 요청  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - KOS RDS 수수료   3. 변경 전 기능(현재 기능 및 문제점 기술)  - 실적 연동 수정, Factor 개발   4. 변경 후 기능(개선 될 내용 기술)  - 비즈나루→RDS 수수료 연동 시, 개통시점의 판매/접점 정보 연동하고 있으나,   사업부서 의 실적인정 운영 기준 상, 당월 내 명변 시, “최종 명의변경” 처리 후의 판매/접점 정보를 연동해야함.  (유무선상품과 달리, 명변후 판매접점/대리점 바뀌는 경우 간혹 발생)  - Internet-기가아이즈 하위에서 하이오더 실적 집계중이며  하이오더(자급제) 구분이 필요할 수도 있는 상황으로,   “단말구분 Factor” 생성 희망함  ============================= DR-2025-30733 [대채기] 비즈나루-RDS 수수료 연동 로직 개선 요청 ㅇ 기존 부가서비스관리 로직 사용 협의 - 정책수수료등로 > 부가서비스관리 > SOIP > '001 글로벌 FREE' 등록 사용 - 정책 추가사항 발생시, 추후 스펙회의 필요",
      "registration_datetime": "2025-05-29 17:56",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널기획팀",
      "registrant": "양혜주",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "비즈나루 프리징으로 인한 개발진행 불가(~25.09)",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-32606",
      "project_title": "001글로벌Free(8.1일 출시 예정) 상품 정산을 위한 RDS 정책수수료 개발",
      "requirements": "1. 요약  - 001글로벌Free(8.1일 출시 예정) 상품 정산을 위한 RDS 정책수수료 개발  ** 유사 상품인 001Free 정산 방식과 유사할 것으로 판단되나, 정책수수료 개발이 필요할지 검토 필요 2. 상세 설명  - 001Free는 KT모바일에서만 가입 가능한 상품으로, 일반 대리점 중, 특정 대리점에 대해서만 정산처리가 되고 있음  - 신규상품(001글로벌Free)는 KT유선(PSTN/SoIP/Centrex), 자타사무선(MVNO포함) 모두 가입 가능하고, 특정 대리점에 정산처리 예정  ==================================================================== DR-2025-32606 001글로벌Free(8.1일 출시 예정) 상품 정산을 위한 RDS 정책수수료 개발 ㅇ 최초개통서비스계약아이디 연동 확인 -신규 개통/해지시만 최초서비스계약아이디를 갱신 -RDS에서 명의변경시 연동이 들어오지 않음을 확인 ->비즈나루 측에서 명의변경 대응건에 대해 개발필요     -비즈나루측 프리징 7~9월까지 프리징 연동개발이 어려운 상황 프리징 이후 협의  ㅇInternet-기가아이즈 “단말구분 Factor” 생성 - Internet-기가아이즈 >요금상품(상품) > 하이오더(자급단말) 상품으로 사용 협의  ㅇ확인사항  -총단말댓수 표기방법 로봇: 1대 하이오더: 메뉴판+알림판+웨이팅단말 총 댓수 기가아이즈(기존연동 건): null",
      "registration_datetime": "2025-06-12 09:46",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "국제전화국",
      "team": "국제전화사업팀",
      "registrant": "이인국",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "보류",
      "previous_month_status": "스펙미비 보류",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33737",
      "project_title": "KT임직원 AIP 암호화 정책으로 인한 조정처리 개선",
      "requirements": "ㅇ 배경: KOS 시스템에서 파일 업로드 시 AIP 암호화 문서 업로드 불가. DRM 문서로 변환하여 업로드하고 있음 ㅇ 현상: 정책지등록>C타입 근거자료 업로드 시 간헐적으로 불가 현상 발생  1. 파일업로드를 위해 UI 파일탐색기에서 엑셀파일(DRM문서) 선택  2. 엑셀 내 Validation체크를 위해 선택된 파일을 읽어 UI 데이터셋에 저장  3. 로컬PC -> RDS 파일서버에 업로드  4. 문서 복호화 (소프트캠프 솔루션)  5. 서비스에서 해당 파일 오픈하여 조정내역 테이블에 저장 (On-Demand 처리) ㅇ 원인: (위 프로세스 중 2번) 선택된 파일를 읽으면서 AIP문서로 자동 변환되며, 해당파일을 파일서버에 올린 후 문서 복호화 불가 (AIP문서의 경우 복호화 불가) ㅇ 개선사항: 파일업로드 시 임시파일로 copy 후 데이터셋에 저장/임시파일 삭제. 파일서버에는 원본파일을 올려서 문서 복호화 처리",
      "registration_datetime": "2025-06-17 17:11",
      "division": "기술혁신부문",
      "headquarters": "IT",
      "department": "Ops본부",
      "team": "고객/유통플랫폼팀",
      "registrant": "김현선",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 2,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33004",
      "project_title": "기업인터넷전화 상품단말 관련 메뉴 기능 개선(주문관리)",
      "requirements": "○ 메뉴명 : K-RDS > 물류 > 주문/배송 > 주문 > 주문등록 ○ 대상단말 : 인터넷전화 ○ 문제점 : 고객본부 기준으로 배정된 단말이 없는 경우 단말 모델명이 미 노출되어 단말 신청이 불가 또는 유통단말이 아닌 단말로 인지 됨 ○ 개선사항  1. 주문 가능 수량이 '0'안 경우에도 주문등록 화면 단말 모델명 노출  2. 안내문구 추가 : 주문가능 수량이 0개 인 경우 담당부서에 확인 요청을 하시기 바랍니다.(사외대리점 > 유통운영팀)  ※담당자 : 황윤섭 책임",
      "registration_datetime": "2025-06-13 14:01",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "커뮤니케이션서비스담당",
      "team": "기업통화서비스팀",
      "registrant": "소남영",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 10,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33353",
      "project_title": "SAP EHP8 업그레이드 프로젝트 2차 연동 전수 테스트",
      "requirements": "1. 요약  - SAP EHP8 업그레이드 프로젝트 2차 연동 전수 테스트(ERP신규서버 &lt;-> RDS개발서버)  2. 테스트 일정  - 사전 공유받은 일정 엑셀파일 첨부",
      "registration_datetime": "2025-06-16 16:48",
      "division": "기술혁신부문",
      "headquarters": "IT",
      "department": "Ops본부",
      "team": "ERP Modernization팀",
      "registrant": "심경언",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "수수료/물류/실적/채권",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-33960",
      "project_title": "와플 관리자페이지 권한 등록 기능 개선",
      "requirements": "1. 요약  - 관리자페이지 권한 등록 시 조직명/직원 이름도 추가 & LIKE 검색 방식 허용  2. 요청 사항  - (현재) 와플 관리자페이지 > 권한관리 > 권한 등록 시, 조직코드나 사번만 입력하게 되어있음  - (개선요청) 조직명/직원 이름도 추가 및 LIKE 검색도 가능하도록 개발 요청",
      "registration_datetime": "2025-06-18 15:58",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "김미선",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-34946",
      "project_title": "기가오피스N 신규상품 개발",
      "requirements": "1. 요약  - 기가오피스N 서비스 출시를 위해 NEOSS~C-SDN 연동 개발 요청  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치  - NEOSS  - C-SDN   3. 신규 기능 상세 설명(추가될 내용 기술)  - C-SDN 연동 개발을 통해서 증속 자동화 반영  - Flexline은 현재 NW시스템 신청일 기준 D+2일 증속 자동화",
      "registration_datetime": "2025-06-24 10:07",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "데이터서비스담당",
      "team": "기업인터넷서비스팀",
      "registrant": "하지인",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-35124",
      "project_title": "[Azure] K지존 Azure전환 관련",
      "requirements": "EAI연동 개발 및 테스트요청,대상  ㅇ NIFI_LWIG_LPRO_E_3001_00 Promotion eMAIL 연동   - 송신: K지존 (WIIF.TB_WEMAIL_TRNRC_INFO)    - 수신: 대량메일발송 (bmail.IF_G_TREAL_QUEUE)    ㅇ NIFI_LWIG_NORD_E_1001_00 가입내역서 정보 연동   - 송신 : K지존 (WIIF.IFT_PRDC_GDNC_MTR_INFO)    - 수신 : KOS-오더 (ORDOWN.AC_PROD_GDNC_MTR_DTL)    ㅇ NIFI_NAUT_LWIG_E_1001_00 사용자 정보   - 송신 : KOS-AUTH (AUTHIF.IFS_USER_BAS1)    - 수신 : K지존 (WIIF.TB_WNBSS_USERS)     ㅇ NIFI_NCDM_LWIG_E_1001_00 상품담당자이력(CDMOWN.PR_PROD_TKCGR_HST)   - 송신 : CDM (CDMOWN.IFS_PR_PROD_TKCGR_HST) / 오더테이블   - 수신 : K지존 (WIIF.tb_wif_prod_tkcgr_info)    ㅇ NIFI_NRDS_LWIG_E_1001_00 기기모델정보(LO_INTM_MODEL_BAS)   - 송신 : KOS-유통 (RDSIF.IFS_LO_INTM_MODEL_BAS)   - 수신 : K지존 (WIIF.TB_WIF_RDS_CD_INFO)",
      "registration_datetime": "2025-06-24 14:14",
      "division": "기술혁신부문",
      "headquarters": "IT",
      "department": "Ops본부",
      "team": "고객/유통플랫폼팀",
      "registrant": "유정재",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-35229",
      "project_title": "[(중요)Windows 11 업그레이드] 대리점 채널 윈도우11 OS 버전 업그레이드 영향도 점검 및 테스트",
      "requirements": "1. 요약  - 25년 10월 14일 윈도우10 OS 버전 업그레이드 종료로 윈도우 11 버전 업데이트 수행전 사전 점검 필요  2. 점검 대상 서비스  ① KOS-오더 : KOS-모바일계약, KOS-모바일상품, KOS-모바일기기  ② K-Note : 대리점경영지원, 통합관리자, 고객신청서스캔  ③ 가치제안서  ④ 사전예약사이트 (KT 샵)  -",
      "registration_datetime": "2025-06-24 16:04",
      "division": "기술혁신부문",
      "headquarters": "IT",
      "department": "Ops본부",
      "team": "고객/유통플랫폼팀",
      "registrant": "이배호",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-35232",
      "project_title": "RDS 내 M2M 단말 분류 추가요청의 건",
      "requirements": "1. 요약 - HIMS 연동 정보 중 NB-IoT(H) 단말 M2M 단말로 추가 2. 요청 기능이 변경되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지) - RDS 3. 변경 전 기능(현재 기능 및 문제점 기술) - 4. 변경 후 기능(개선 될 내용 기술) - HIMS 연동 정보 중 NB-IoT(H) 단말 M2M 단말로 추가  5. 담당자: 황윤섭 책임(유통서비스팀)",
      "registration_datetime": "2025-06-24 16:05",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "기업무선/IoT인프라서비스팀",
      "registrant": "차정하",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-35684",
      "project_title": "(DR-2024-6996/ 기한 만료에 따른 재요청)무선 DCB 부가서비스 생성 요청의 건(유료)",
      "requirements": "1. 요약  - 무선 DCB 부가서비스 생성 요청(유료 상품) 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - KOS B2C > 오더, 빌링 / 마이케이티(고객 서비스가입내역, 청구 항목 등 확인) 3. 신규 기능 상세 설명(추가될 내용 기술)  - 부가서비스 : KT 콘텐츠페이 월 구독 부가서비스  . [서비스] KT 콘텐츠페이 콘텐츠 이용료 구독 멤버십  : 월 구독료 2가지 상품  : 기본형(3,900원) 콘텐츠 이용료 KT 후불결제 5% 할인(최대 1.5만원)  : 프리미엄형(9,900원) 콘텐츠 이용료 KT 후불결제 5% 할인(최대 3만원 + 1만원/ 60만원 이상 결제 시 1만원 추가 할인 제공)  : 할인은 KT 청구서 청구할인 반영(RCM)  . [가입] KOS 전산 가입(오더, 빌링), 마이케이티(고객 확인용)",
      "registration_datetime": "2025-06-26 10:07",
      "division": "Customer부문",
      "headquarters": "서비스Product본부",
      "department": "신규서비스담당",
      "team": "모바일마켓서비스팀",
      "registrant": "곽은경",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-35712",
      "project_title": "기가오피스 N상품 출시에 따른 공통성 기능 개발",
      "requirements": "- 선납 기능 (기가오피스 계약기준 선납, 청구계약선납)  - 상세조회 항목 확인  - 미완료오더처리 그리드 항목 구성  - 상품추가에 따른 공통성 기능 개발 (전체 공통기능 점검)",
      "registration_datetime": "2025-06-26 11:18",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "데이터서비스담당",
      "team": "기업인터넷서비스팀",
      "registrant": "하지인",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-37707",
      "project_title": "SA대리점 수수료 관리를 위한 연동개발(K-ICIS 오더 계약원부)",
      "requirements": "1. 요약  - 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 3. 신규 기능 상세 설명(추가될 내용 기술)  -SA대리점 KRDS 수수료 지급 프로세스 개발을 위한 ICIS 오더 계약 정보 추가 연동",
      "registration_datetime": "2025-07-07 09:57",
      "division": "Enterprise부문",
      "headquarters": "법인사업본부",
      "department": "법인사업1담당",
      "team": "법인사업혁신팀",
      "registrant": "강병진",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 3,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-38257",
      "project_title": "[MVNO] OSST 개통 시 판매정책 및 판재가 조회관련 개선 개발 요청의 건",
      "requirements": "1. 요약  - [MVNO] OSST 개통 시 판매정책 및 판재가 조회관련 개선 개발 요청의 건 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 처리조직ID만 가지고 판매정책 및 판매가 조회 및 반영 4. 변경 후 기능(개선 될 내용 기술)  - 처리조직ID이 없을 경우, 최상위 조직ID(AA?????, HA????? 등 신규사업자 생성시 K-RDS에 등록되는 조직)으로 판매정책 및 판매가 조회 및 반영하도록 개선 개발",
      "registration_datetime": "2025-07-08 16:25",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "MVNO채널담당",
      "team": "MVNO제휴팀",
      "registrant": "함윤배",
      "project_category": "S - 필수 대외과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-39002",
      "project_title": "스마트렌탈 &gt; 정책수수료대상",
      "requirements": "1. 정책수수료대상에 항목은 있으나 자료가 없어 정산자동화 및 검증에 어려움  - 판매정책ID, 판매정책명, 단말모델명 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 수수료>수수료관리>정책수수료대상조회 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 3개 항목 자료 전체 공란 4. 변경 후 기능(개선 될 내용 기술)  - 해당 항목 정보 작성",
      "registration_datetime": "2025-07-11 11:14",
      "division": "재무실",
      "headquarters": "재무서비스센터",
      "department": "유통정산팀",
      "team": "김영자",
      "registrant": "C - 주요 개선 과제",
      "project_category": "KOS-유통",
      "target_system": "이월",
      "current_month_status": "15",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "6월확정과제",
      "project_id": "DR-2025-38979",
      "project_title": "SA대리점 수수료 관리를 위한 연동개발(BIDW 기업지능망",
      "requirements": "기업메시징인 상품의 트래픽 정보),1. 요약  - 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 4. 변경 후 기능(개선 될 내용 기술)  -L3가 기업지능망,기업메시징인 상품중에서 트래픽이 미발생한 경우 수수료 환수",
      "registration_datetime": "2025-07-11 10:45",
      "division": "Enterprise부문",
      "headquarters": "법인사업본부",
      "department": "법인사업1담당",
      "team": "법인사업혁신팀",
      "registrant": "강병진",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "신규",
      "previous_month_status": "",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-08403",
      "project_title": "RDS '정책수수료 대상' 화면 내 부가서비스 관련 정책수수료 등록 화면 개선요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-02-12 17:02",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 6,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "C_01",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-21754",
      "project_title": "인터넷 TV 업셀링 정책 관련 자동화 팩터 개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-14 09:46",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선케어정책팀",
      "registrant": "임명섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "중기",
      "effort_hours": 35,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "C_02",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-08410",
      "project_title": "KOS RDS 내 정책 실적 등록 시 '필수조건 동시복사' 기능 개발요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-02-12 17:10",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 12,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "C_03",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30515",
      "project_title": "본인확인서비스 사이트 코드 변경 개발의 건",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-29 10:23",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선케어정책팀",
      "registrant": "김보라",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 3,
      "unit_system": "유통망",
      "previous_month_status": "신규",
      "remarks": "C_04",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-27080",
      "project_title": "단말원부 등록 중복업로드 방지",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-08 14:19",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "손석민",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 8,
      "unit_system": "물류",
      "previous_month_status": "이월",
      "remarks": "C_05",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-31680",
      "project_title": "[개발_전산] 대외_행안부모바일신분증 진위확인 검증체계 고도화",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-09 10:25",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "송지숙",
      "project_category": "S - 필수 대외과제",
      "target_system": "KOS-유통",
      "current_month_status": "취소",
      "effort_hours": 0,
      "unit_system": "유통망",
      "previous_month_status": "유통망",
      "remarks": "신규",
      "priority": "RDS 개발사항 없음"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30308",
      "project_title": "대리점 운영자금 이자 청구 기능 반영 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-28 11:15",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 15,
      "unit_system": "채권",
      "previous_month_status": "신규",
      "remarks": "C_07",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-28810",
      "project_title": "[애플] 애플워치 GPS모델 RMA 등록",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-21 09:13",
      "division": "Customer부문",
      "headquarters": "Device사업본부",
      "department": "Device마케팅담당",
      "team": "Device사업팀",
      "registrant": "원정섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 10,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "C_08",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-28917",
      "project_title": "전화투표 서비스 수수료 지급 방식 변경",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-21 14:06",
      "division": "Customer부문",
      "headquarters": "소상공인사업본부",
      "department": "소상공인상품담당",
      "team": "소상공인통화상품팀",
      "registrant": "이국희",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 15,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "수수료 공수 부족",
      "priority": "C_09"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-25065",
      "project_title": "[RDS]애플 액세서리 출고 실적=판매실적 연동",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-29 16:24",
      "division": "Customer부문",
      "headquarters": "Device사업본부",
      "department": "Device마케팅담당",
      "team": "Device사업팀",
      "registrant": "원정섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 10,
      "unit_system": "실적",
      "previous_month_status": "이월",
      "remarks": "C_10",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-22014",
      "project_title": "소상공인 신규 제휴카드 연동개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-14 18:46",
      "division": "Customer부문",
      "headquarters": "소상공인사업본부",
      "department": "소상공인상품담당",
      "team": "소상공인상품제휴팀",
      "registrant": "배동훈",
      "project_category": "A - 필수 전략과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "실적",
      "previous_month_status": "실적",
      "remarks": "보류",
      "priority": "사업부서 요청 보류"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30656",
      "project_title": "[개발요청] 대리점 유선상품 사업용회선 해지를 위한 프로세스 개선",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-29 15:19",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "강영주",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 17,
      "unit_system": "유통망",
      "previous_month_status": "신규",
      "remarks": "C_12",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-29978",
      "project_title": "정리점 강제수납 제한 해제 개발요청2",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-27 11:14",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "김준",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 7,
      "unit_system": "채권/유통망",
      "previous_month_status": "신규",
      "remarks": "오더 동시개발 필요",
      "priority": "C_13"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-26269",
      "project_title": "모델별 배정현황 신규메뉴 개발 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-08 14:19",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "한민구",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "중기",
      "effort_hours": 15,
      "unit_system": "물류",
      "previous_month_status": "이월",
      "remarks": "공수부족",
      "priority": "C_14"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-27850",
      "project_title": "[추가] M&S 통합개통센터 전국확대 (기존 개발: M&S 통합개통 센터 관련 KRDS 조직관계 신설 검토 요청의 건)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-15 16:08",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널기획팀",
      "registrant": "강혜영",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 1,
      "unit_system": "유통망",
      "previous_month_status": "신규",
      "remarks": "C_15",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-03945",
      "project_title": "KTS 업셀링을 위한 정산 고도화",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-01-20 15:45",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선유통혁신팀",
      "registrant": "김세아",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "보류",
      "priority": "스펙미비(8월 스펙회의 진행 예정)"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-08407",
      "project_title": "KOS RDS 내 '대리점별 집계'화면 내 연계단말 처리 유통조직(대리점) 정보 추가요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-02-12 17:05",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "수수료 공수 부족",
      "priority": "C_18"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30836",
      "project_title": "결산집계표종류 분류 개선 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-30 10:09",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 8,
      "unit_system": "채권",
      "previous_month_status": "신규",
      "remarks": "C_19",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-26261",
      "project_title": "배정등록 메뉴 배정가능 표기값 로직 변경 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-08 14:01",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "한민구",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "물류",
      "previous_month_status": "이월",
      "remarks": "공수부족",
      "priority": "C_20"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-27430",
      "project_title": "[유통/채권?mRDS] mRDS '입금처리' 화면 상계지급유보 입금시 default입금일 수정요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-14 09:15",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "김준",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 8,
      "unit_system": "와플/채권",
      "previous_month_status": "신규",
      "remarks": "C_21",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-20613",
      "project_title": "RDS 내 정책별집계 내 전체지급취소 환수여부 표기",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-08 10:46",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 25,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "수수료 공수 부족",
      "priority": "C_22"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32526",
      "project_title": "Watchdog 점검 지표에 신규 지표 추가 요청의 건(유통점 청구주소/납부자여부)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-11 16:44",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "구도윤",
      "project_category": "A - 필수 전략과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 20,
      "unit_system": "실적",
      "previous_month_status": "신규",
      "remarks": "C_23",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-12778",
      "project_title": "마케팅부 지사 표기",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-05 11:58",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "손석민",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "물류",
      "previous_month_status": "이월",
      "remarks": "공수부족",
      "priority": "C_24"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-29577",
      "project_title": "[통합개통센터] 유무선 실적 확인을 위한 전산개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-26 10:14",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널기획팀",
      "registrant": "강혜영",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "완료",
      "effort_hours": 0,
      "unit_system": "유통망/실적",
      "previous_month_status": "유통망/실적",
      "remarks": "신규",
      "priority": "C_25"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-12287",
      "project_title": "코코넛센터 주문 대리점 여신부족 데이터 연동 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-02-28 18:07",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "한민구",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 20,
      "unit_system": "채권",
      "previous_month_status": "이월",
      "remarks": "공수부족",
      "priority": "C_26"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-28359",
      "project_title": "단말 원부 초기화양식 변경",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-19 17:24",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "한민구",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "C_27",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-09797",
      "project_title": "K-RDS 정책수수료 Grade 단가 환수 기능 보완 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-02-18 16:59",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 60,
      "unit_system": "수수료",
      "previous_month_status": "보류",
      "remarks": "수수료 공수 과다",
      "priority": "C_28"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-20615",
      "project_title": "2G 가입자 5G 전환신규 처리 불가오류 가능하도록 개발요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-08 10:48",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "보류",
      "priority": "오더 개발필요"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-20617",
      "project_title": "KOS-RDS 실사용일수 관점 추가",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-08 10:50",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 30,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "수수료 공수 부족",
      "priority": "C_30"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-27337",
      "project_title": "RDS 시점재고 메뉴의 재고조정 버튼 숨김 처리",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-13 15:23",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "김태현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "물류",
      "previous_month_status": "신규",
      "remarks": "C_31",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-16511",
      "project_title": "정책단가 코드합산 및 계약점 적용 개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-20 13:10",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "이영우",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 30,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "수수료 공수 부족",
      "priority": "C_32"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-16148",
      "project_title": "[개발요청] 유선상품 사업용회선 해지를 위한 프로세스 개선",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-19 11:22",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "사업지원담당",
      "team": "인터넷사업지원팀",
      "registrant": "김홍수",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 17,
      "unit_system": "유통망",
      "previous_month_status": "이월",
      "remarks": "C_33",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32048",
      "project_title": "지니TV 신규상품 출시 개발 요청 건",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-10 10:27",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "요금혁신담당",
      "team": "유선요금혁신팀",
      "registrant": "김민화",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "신규",
      "priority": "C_34"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32321",
      "project_title": "와플_무선영업현황 및 RDS_(조직별)영업계약실적 조직명 변경",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-11 09:45",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "무선정책담당",
      "team": "무선정책기획팀",
      "registrant": "송시욱",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "실적",
      "previous_month_status": "신규",
      "remarks": "C_35",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2024-71529",
      "project_title": "신규 폰 렌탈 사업 추진",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2024-12-16 11:19",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "사업지원담당",
      "team": "무선사업지원팀",
      "registrant": "김현수",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "물류/채권",
      "previous_month_status": "물류/채권",
      "remarks": "보류",
      "priority": "스펙미비 보류"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2024-71725",
      "project_title": "중고 폰 렌탈 사업 추진",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2024-12-16 17:06",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "사업지원담당",
      "team": "무선사업지원팀",
      "registrant": "김현수",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "물류/채권",
      "previous_month_status": "물류/채권",
      "remarks": "보류",
      "priority": "스펙미비 보류"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2024-72336",
      "project_title": "[W 프로젝트] 모바일 초이스 3종 신규 출시",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2024-12-18 17:10",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "요금혁신담당",
      "team": "무선요금혁신팀",
      "registrant": "최상균",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "보류",
      "remarks": "오더 개발시 비바체 배포",
      "priority": "C_38"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-20567",
      "project_title": "W 초이스 후속 개발 (외부 연동/상품 상세)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-08 10:09",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "요금혁신담당",
      "team": "무선요금혁신팀",
      "registrant": "최상균",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "보류",
      "remarks": "오더 개발시 비바체 배포",
      "priority": "C_39"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-04798",
      "project_title": "KOS-RDS 와치독 기능 개발_IoT",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-01-22 16:27",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "IoT서비스1팀",
      "registrant": "김해건",
      "project_category": "A - 필수 전략과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "실적",
      "previous_month_status": "실적",
      "remarks": "보류",
      "priority": "통합 고도화 진행 예정"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-09413",
      "project_title": "[제휴부가]신규 CP 제휴 부가서비스 'AI 범죄차단 서비스' 개발 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-02-17 16:59",
      "division": "Customer부문",
      "headquarters": "서비스Product본부",
      "department": "신규서비스담당",
      "team": "결제인증서비스팀",
      "registrant": "황보영",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-15160",
      "project_title": "타사고객 번호이동 후 RDS 리워드 사용내역 누락 개선 ",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-14 10:00",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "임세인",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "와플/유통망",
      "previous_month_status": "와플/유통망",
      "remarks": "이월",
      "priority": "개발 진행 시 CRM하고 협의 필요"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-17873",
      "project_title": "[개발요청] RDS 내 대리점 OA 및 사업용회선 등 자산관리 화면 반영 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-26 15:51",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "강영주",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "완료",
      "effort_hours": 7,
      "unit_system": "유통망",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-17907",
      "project_title": "기가오피스 신규 상품 전산 개발 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-26 16:29",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "데이터서비스담당",
      "team": "기업인터넷서비스팀",
      "registrant": "하지인",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "오더 개발시 진행",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-18087",
      "project_title": "RDS 전자계약관리 상 불필요 기능 제거 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-27 10:33",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "채권",
      "previous_month_status": "채권",
      "remarks": "보류",
      "priority": "AECS개발시 개발"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-18088",
      "project_title": "RDS 전자계약 보관 기능 이관 및 권한 확인 연동 기능 구현 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-27 10:34",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "유통망/채권",
      "previous_month_status": "유통망/채권",
      "remarks": "보류",
      "priority": "AECS개발시 개발"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-18089",
      "project_title": "RDS 전자계약관리 상 매장계약 기능 추가 개발 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-03-27 10:34",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "유통망/채권",
      "previous_month_status": "유통망/채권",
      "remarks": "보류",
      "priority": "AECS개발시 개발"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-20340",
      "project_title": "[요금제 수정] IoT 안심결제 요금제 스펙 수정 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-07 14:28",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "기업무선/IoT결제서비스팀",
      "registrant": "구대환",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "이월",
      "remarks": "오더 개발시 진행",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-21487",
      "project_title": "“단골고객” 이관 시 발송 문자 SRTT 적재되도록 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-11 10:30",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "김혜옥",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 7,
      "unit_system": "유통망",
      "previous_month_status": "이월",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-21871",
      "project_title": "KOS-RDS상에 업무처리조직 순서 변경 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-14 13:42",
      "division": "부산/경남광역본부",
      "headquarters": "부산/경남고객본부",
      "department": "창원지사",
      "team": "마케팅부",
      "registrant": "손강희",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "유통망",
      "previous_month_status": "유통망",
      "remarks": "보류",
      "priority": "개발불가"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-24658",
      "project_title": "안심번호 SA대리점 위탁상품 적용 수수료 추가 개발(담당자:나혜정부장님)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-04-28 16:02",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "커뮤니케이션서비스담당",
      "team": "기업지능망서비스팀",
      "registrant": "전미옥",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "보류",
      "priority": "개발 가능 여부 검토중 "
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-27403",
      "project_title": "대출원장 지급일자 항목 추가 및 SAP 연동 개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-13 17:56",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 6,
      "unit_system": "채권",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-27901",
      "project_title": "RDS  (대리점 운영자금 옵션을 이자청구로 변경)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-15 18:14",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널관리팀",
      "registrant": "이상현",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 20,
      "unit_system": "채권",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32023",
      "project_title": "여주물류센터 배정제외상품 본부별 배정 기능 개선 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-10 09:44",
      "division": "Customer부문",
      "headquarters": "Device사업본부",
      "department": "Device마케팅담당",
      "team": "Device사업팀",
      "registrant": "황진주",
      "project_category": "프로젝트 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "물류",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32260",
      "project_title": "본인확인서비스 미점유인증-점유인증 전환",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-10 18:06",
      "division": "Customer부문",
      "headquarters": "마케팅혁신본부",
      "department": "유선정책담당",
      "team": "유선케어정책팀",
      "registrant": "김보라",
      "project_category": "S - 필수 대외과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 3,
      "unit_system": "유통망",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-28426",
      "project_title": "M포탈 조직스위치 기능 구현",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-20 09:03",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "김미정",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "취소",
      "effort_hours": 0,
      "unit_system": "유통망",
      "previous_month_status": "유통망",
      "remarks": "신규",
      "priority": "RDS 개발사항 없음"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-28594",
      "project_title": "IoT통합약정 개발 요청 ",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-20 15:07",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "기업무선/IoT결제서비스팀",
      "registrant": "구대환",
      "project_category": "B - 필수 개선과제",
      "target_system": "KOS-유통",
      "current_month_status": "취소",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "신규",
      "priority": "RDS 개발사항 없음"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30070",
      "project_title": "RDS 유통망정보 &gt; CRM운영여부 조건값 개발_수정",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-27 15:20",
      "division": "Customer부문",
      "headquarters": "AX혁신지원본부",
      "department": "AX역량혁신담당",
      "team": "유통역량혁신팀",
      "registrant": "김미선",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 3,
      "unit_system": "유통망",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30504",
      "project_title": " IoT 대용량 QoS요금제 6종 개발(LTE/5G)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-29 09:53",
      "division": "Enterprise부문",
      "headquarters": "서비스Product본부",
      "department": "기업무선/IoT서비스담당",
      "team": "기업무선/IoT관제서비스팀",
      "registrant": "임종섭",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "이월",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "신규",
      "priority": "오더 개발시 진행"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-30733",
      "project_title": "[대채기] 비즈나루-RDS 수수료 연동 로직 개선 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-05-29 17:56",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "대면채널혁신담당",
      "team": "대면채널기획팀",
      "registrant": "양혜주",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "신규",
      "priority": "비즈나루 프리징으로 인한 개발진행 불가(~25.09)"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32090",
      "project_title": "국제전화 001글로벌Free 출시(25.7.1) 위한 K-RDS 판매자 등록 개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-10 12:55",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "국제전화국",
      "team": "국제전화사업팀",
      "registrant": "이인국",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 5,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32331",
      "project_title": "[MVNO] 신규사업자 GME 일반 후불 RS 요금제 4종 개발 요청",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-11 09:57",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "MVNO채널담당",
      "team": "MVNO채널개발팀",
      "registrant": "이정학",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "취소",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "신규",
      "priority": "RDS 개발사항 없음"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32606",
      "project_title": "001글로벌Free(8.1일 출시 예정) 상품 정산을 위한 RDS 정책수수료 개발",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-12 09:46",
      "division": "Customer부문",
      "headquarters": "영업·채널본부",
      "department": "국제전화국",
      "team": "국제전화사업팀",
      "registrant": "이인국",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "보류",
      "effort_hours": 0,
      "unit_system": "수수료",
      "previous_month_status": "수수료",
      "remarks": "신규",
      "priority": "스펙미비 보류"
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2025-32834",
      "project_title": "SA대리점 수수료 관리를 위한 연동개발(BIDW 관리고객ID)",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2025-06-12 17:30",
      "division": "Enterprise부문",
      "headquarters": "법인사업본부",
      "department": "법인사업1담당",
      "team": "법인사업혁신팀",
      "registrant": "강병진",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 3,
      "unit_system": "수수료",
      "previous_month_status": "신규",
      "remarks": "",
      "priority": ""
    },
    {
      "project_month": "7월확정과제",
      "project_id": "DR-2024-66164",
      "project_title": "스마트렌탈 정책 수수료 지급 후 익월에 사은품 개통 취소 시 환수 및 일반정책 지급",
      "requirements": "비즈니스 요구사항",
      "registration_datetime": "2024-11-18 09:37",
      "division": "Customer부문",
      "headquarters": "영업본부",
      "department": "유선영업담당",
      "team": "유선유통운영팀",
      "registrant": "백지원",
      "project_category": "C - 주요 개선 과제",
      "target_system": "KOS-유통",
      "current_month_status": "개발",
      "effort_hours": 10,
      "unit_system": "수수료",
      "previous_month_status": "중기",
      "remarks": "",
      "priority": ""
    }
]


## Upload the documents

In [138]:
# Upload documents to the index
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT,
                      index_name=AZURE_SEARCH_INDEX,
                      credential=search_credential)
try:
    result = search_client.upload_documents(documents=documents)
    for r in result:
        print(f"Key: {r.key}, Succeeded: {r.succeeded}, ErrorMessage: {r.error_message}")
except Exception as ex:
    print("Failed to upload documents:", ex)


    index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT, credential=search_credential)

Key: DR-2025-33428, Succeeded: True, ErrorMessage: None
Key: DR-2025-20567, Succeeded: True, ErrorMessage: None
Key: DR-2025-03945, Succeeded: True, ErrorMessage: None
Key: DR-2025-33426, Succeeded: True, ErrorMessage: None
Key: DR-2025-34209, Succeeded: True, ErrorMessage: None
Key: DR-2025-38657, Succeeded: True, ErrorMessage: None
Key: DR-2025-35859, Succeeded: True, ErrorMessage: None
Key: DR-2025-37608, Succeeded: True, ErrorMessage: None
Key: DR-2025-29978, Succeeded: True, ErrorMessage: None
Key: DR-2025-22014, Succeeded: True, ErrorMessage: None
Key: DR-2025-34412, Succeeded: True, ErrorMessage: None
Key: DR-2025-33896, Succeeded: True, ErrorMessage: None
Key: DR-2025-32526, Succeeded: True, ErrorMessage: None
Key: DR-2025-34376, Succeeded: True, ErrorMessage: None
Key: DR-2025-08407, Succeeded: True, ErrorMessage: None
Key: DR-2025-20613, Succeeded: True, ErrorMessage: None
Key: DR-2025-27337, Succeeded: True, ErrorMessage: None
Key: DR-2025-12778, Succeeded: True, ErrorMessag

In [139]:
# Run an empty query (returns selected fields, all documents)
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='project_title,requirements,current_month_status',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["project_title"])
    print(result["current_month_status"])
    print(f"requirements: {result['requirements']}")


Total Documents Matching Query: 96
1.0
SA대리점 수수료 관리를 위한 연동개발(BIDW 기업지능망
보류
requirements: 기업메시징인 상품의 트래픽 정보),1. 요약  - 2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - 3. 변경 전 기능(현재 기능 및 문제점 기술)  - 4. 변경 후 기능(개선 될 내용 기술)  -L3가 기업지능망,기업메시징인 상품중에서 트래픽이 미발생한 경우 수수료 환수
1.0
인터넷 TV 업셀링 정책 관련 자동화 팩터 개발
중기
requirements: 비즈니스 요구사항
1.0
단말원부 등록 중복업로드 방지
개발
requirements: 비즈니스 요구사항
1.0
[개발_전산] 대외_행안부모바일신분증 진위확인 검증체계 고도화
취소
requirements: 비즈니스 요구사항
1.0
전화투표 서비스 수수료 지급 방식 변경
이월
requirements: 비즈니스 요구사항
1.0
KTS 업셀링을 위한 정산 고도화
보류
requirements: 비즈니스 요구사항
1.0
코코넛센터 주문 대리점 여신부족 데이터 연동 요청
이월
requirements: 비즈니스 요구사항
1.0
지니TV 신규상품 출시 개발 요청 건
이월
requirements: 비즈니스 요구사항
1.0
W 초이스 후속 개발 (외부 연동/상품 상세)
보류
requirements: 비즈니스 요구사항
1.0
M포탈 조직스위치 기능 구현
취소
requirements: 비즈니스 요구사항
1.0
[대채기] 비즈나루-RDS 수수료 연동 로직 개선 요청
보류
requirements: 비즈니스 요구사항
1.0
수도권 도매채널 거래관계생성 BPM품의 호출조건 변경_특정 판매점 리스트 BPM품의 강제호출 적용의 건
개발
requirements: 1. 요약  - 판매점 조직관계 신규 > 미책임 대리점과 거래관계생성 시 BPM 호출  - 특정 접점코드의 경우 무조건 BPM 호출  2. 요청 기능이 추가되어야 할 서비스

In [ ]:
# Run a term query
results =  search_client.search(query_type='simple',
    search_text="개발" ,
    select='project_month,project_title,requirements,current_month_status',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {} {}".format(result["project_month"], result["project_title"], result["requirements"]))

Total Documents Matching Query: 32
1.1631508
6월확정과제: [대채기] RDS 정책수수료등록 하이오더 실적집계로직 개선개발 1. 요약  - 하이오더 실적집계 로직 개선 (현재 하이오더는 인터넷 하위에서 집계되고 있음)  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - RDS 정책수수료등록  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 하이오더 총단말댓수 (메뉴판+알림판+웨이팅단말 포함) 이 곧 실적인데, 현재는 1고객계약이 곧 1개의 실적으로 잡히고 있음.  4. 변경 후 기능(개선 될 내용 기술)  DR-2024-32861 개발 건 후속으로,  RDS 정책수수료대상조회 에서는 “카메라 대수”실적 로 정상적으로 잘 전송되는 것을 확인하였으나,  정작 B타입 수수료 등록시에는, 대리점별집계 엑셀다운로드로 확인하여 보니, “사업자당/고객당 실적”으로 잡히는 것으로 보입니다. Ex: 피앤에프 대리점삼거리점은 하이오더 실적 28건 (20건+8건) 을 인정받아야하나, 대리점별 집계로보면 2건으로 인정됩니다.   &lt;실적집계방식 개선제언/유통서비스팀> - 카메라댓수*실적건수 하는 방식으로 총단말댓수 팩터에 가중치 적용
1.060872
6월확정과제: 수도권 도매채널 거래관계생성 BPM품의 호출조건 변경_특정 판매점 리스트 BPM품의 강제호출 적용의 건 1. 요약  - 판매점 조직관계 신규 > 미책임 대리점과 거래관계생성 시 BPM 호출  - 특정 접점코드의 경우 무조건 BPM 호출  2. 요청 기능이 추가되어야 할 서비스의 메뉴 위치(ex. KATE > Notice > 전사공지)  - RDS > 유통망 > 유통망관리 > 유통망정보관리  3. 변경 전 기능(현재 기능 및 문제점 기술)  - 신규관계설정이 아래 조건 중 하나 이상 만족 시에만 BPM 관계설정품의 호출  (1) 수도권 도매영업부소속 대리점과 맺은 관계설정이 판매점 권역에 설정된 N건 초과 시 호출  (2) 해당

In [107]:
results = search_client.search(
    search_text="개발", 
    search_fields=['current_month_status'], 
    select='current_month_status,project_title,requirements',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print("{}: {} {}".format(result["current_month_status"], result["project_title"], result["requirements"]))

Total Documents Matching Query: 31
개발: 국제전화 001글로벌Free 출시(25.7.1) 위한 K-RDS 판매자 등록 개발 비즈니스 요구사항
개발: 결산집계표종류 분류 개선 요청 비즈니스 요구사항
개발: 하이오더/로봇 RDS > C타입(수기) 정산을 위한 개발 요청 ㅇ C부문 감사 조치 요구사항   1)획득-유지판촉비 계정 혼용 불가  2) 획득 세금계산서 정산 축소  3) 획득 정산 시 정책지 등에 정산 근거자료 필수 ㅇ 현재 하이오더와 로봇은 CONAN 전산을 통해 하이오더/로봇 모집수수료를 RDS 전송하는 형태로 운영중이며, BM외 수수료 로딩 권한이 없어 RDS 정산 개발을 통한 고객본부별 정산 기능 부여/관리 및 정책지 근거자료 등록, 로딩을 통한 투명한 정책 수수료 운영 목적 ㅇ 요청사항  - RDS > 수수료 > 정책지등록 >상품에 하이오더와 로봇 개발 필요(미래성장 > 하이오더 or 로봇 > 일반)  1차로 C타입(수동) 개발 후 완료 시 A,B타입(자동) 개발 예정   - 수수료코드 개발 필요(수수료코드세팅) : 하이오더 정책수수료, 로봇 정책수수료 등  . 현재 수수료 코드(하이오더 E106, 로봇 E104) 정책수수료 수용 가능한지 확인 후 필요 시 개발 진행 필요    DR-2025-35859  하이오더/로봇 RDS > C타입(수기) 정산을 위한 개발 요청   스펙회의   C타입 우선 개발 1. 우선순위에 따라 SR로 진행가능 2. SR로 개발시 LOB 미분리, 인터넷 하위에 포함 3. 데이터 확인 및 권한 필요   ->지급대상이 현재 정책수수료대상조회(원부)에 포함되는지   ->정책수수료대상조회 인터넷-기가아이즈 데이터 유통정산 팀에 요청 4. 추후에 A/B타입 개발시 마이그레이션 필요  추후 A/B 타입 개발시 필요 확인 사항 1. 사업화필요 2. LOB 분리 여부 3. 필요팩터 정리 4. 향후 사업화 규모 확인시 정책지 제공 필요 5. 사업화 준비 단계에서 추가 스펙회의 필요
개발: [추가] M&S 통합개

In [108]:
# Add a filter
results = search_client.search(
    search_text="개발", 
    select='project_id,project_title,requirements,current_month_status,effort_hours', 
    filter='effort_hours gt 10', 
    order_by='effort_hours desc',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print("{}: [{}] {} {}".format(result["current_month_status"], result["effort_hours"], result["project_id"],result["project_title"], result["requirements"]))

Total Documents Matching Query: 12
중기: [35] DR-2025-21754 인터넷 TV 업셀링 정책 관련 자동화 팩터 개발
이월: [30] DR-2025-16511 정책단가 코드합산 및 계약점 적용 개발
개발: [23] DR-2025-37608 [대채기] RDS 정책수수료등록 하이오더 실적집계로직 개선개발
개발: [20] DR-2025-27901 RDS  (대리점 운영자금 옵션을 이자청구로 변경)
개발: [17] DR-2025-30656 [개발요청] 대리점 유선상품 사업용회선 해지를 위한 프로세스 개선
개발: [15] DR-2025-35859 하이오더/로봇 RDS > C타입(수기) 정산을 위한 개발 요청
중기: [15] DR-2025-26269 모델별 배정현황 신규메뉴 개발 요청
개발: [15] DR-2025-30308 대리점 운영자금 이자 청구 기능 반영 요청
중기: [15] DR-2025-38657 [디바이스단독]통신향 공단말의 삼디초 요금제 적용 과제
중기: [15] DR-2025-34412 eSIM 탑재 재생단말 수용을 위한 기등록 단말 원부 변경 조치 메뉴 개발 요청
개발: [12] DR-2025-08410 KOS RDS 내 정책 실적 등록 시 '필수조건 동시복사' 기능 개발요청
개발: [12] DR-2025-33426 수도권 도매채널 거래관계생성 BPM품의 호출조건 변경_특정 판매점 리스트 BPM품의 강제호출 적용의 건


In [117]:
# Run a term query
results =  search_client.search(query_type='semantic',
    semantic_configuration_name="my-semantic-config",
    search_text="6월" ,
    select='project_id,project_title,requirements,current_month_status,effort_hours',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: [{}] {} {}".format(result["current_month_status"], result["effort_hours"], result["project_id"],result["project_title"], result["requirements"]))

Total Documents Matching Query: 0


## Create the vector query string

In [40]:
### Query is 1536-dimensional vector of "quintessential lodging near running trails, eateries, retail"
### Note: this same vector can be used for vector search and semantic search
### Replace with your own vector
vector =  [-0.045507785,0.028645637,0.014222746,-0.018325701,-0.020214563,-0.038177505,0.015761355,0.047784425,0.010457533,-0.042280458,0.046658613,0.0010140118,0.008381038,-0.009988446,0.0053694933,0.05784167,0.004900405,0.011414473,0.037527036,0.08145868,0.0048034606,-0.036801513,-0.059943184,-0.020614851,-0.01619917,0.032973755,-0.03532545,-0.013622314,0.009012743,-0.010657678,-0.03354917,-0.041229703,0.004687752,-0.09882119,0.057391346,0.019413985,-0.010832804,-0.010069754,0.031922996,-0.0033805603,0.010926622,0.0031381983,0.048660055,-0.0047846967,0.011595854,0.001674644,0.03645126,0.034374762,-0.030922277,-0.012765447,-0.01850083,-0.053588606,-0.00835602,-0.06674808,-0.013834967,0.008368528,0.01100793,-0.004475099,0.07610483,0.0130844265,0.00012381967,-0.0016246078,0.013859984,0.049685795,0.023642031,0.042455584,-0.008443583,0.024104865,-0.055990335,-0.015673792,0.009100306,-0.03972862,-0.043931648,0.0052350215,0.06809906,-0.0184633,-0.003202307,0.0057729087,0.009606921,-0.018625919,0.0040091383,0.016599458,-0.0022719493,-0.004809715,-0.0045595346,-0.052087523,-0.041980244,-0.000460488,-0.034574907,0.048359837,0.03342408,-0.014598017,0.026343979,-0.058291994,-0.016737057,0.0073052626,0.020539798,-0.010194845,-0.033499133,-0.014635543,0.037502017,-0.089964814,0.0035807046,-0.037176784,-0.0020061329,-0.0072990083,0.024117375,0.02090256,0.022853965,-0.0027723096,-0.0719018,0.02684434,0.010957894,0.024254974,-0.039953783,0.055740155,-0.01708731,-0.016962219,0.01481067,0.05934275,0.019701693,0.021102702,-0.008024531,-0.035150323,-0.00784315,-0.042105332,-0.04695883,-0.014410381,-0.056740876,-0.04115465,0.0025393295,0.02847051,0.020552306,0.01456049,-0.034224655,-0.017149854,-0.015923971,-0.02254124,-0.041054577,-0.00031116165,-0.00522564,-0.015798882,0.011233092,-0.027669935,0.014535472,0.020777468,0.019914346,0.017762797,0.017537635,0.040354073,0.0073115174,-0.012790465,-0.0087375445,0.009544376,-0.06434636,-0.013222026,-0.024079848,-0.019964382,-0.024530172,0.023979776,-0.055740155,-0.024830388,-0.016549421,0.03770216,0.020152017,-0.049185432,-0.020402199,0.041304756,-0.074503675,-0.050211173,0.06669805,0.0069675194,-0.035875846,0.06894967,-0.0031585253,0.0018700972,0.0086062,-0.0059386534,-0.02696943,-0.007793114,0.0016246078,-0.04463215,-0.0043687723,0.031922996,0.03975364,0.06309546,-0.035900865,0.015160922,-0.037276853,0.011752216,-0.04795955,0.00068017753,-0.002251622,0.003399324,0.044982407,0.01107673,-0.017012255,0.01045128,0.010207353,0.027419753,-0.060293436,-0.02139041,0.028745709,-0.0027300918,-0.03987873,-0.032323286,0.011764726,-0.015873935,-0.034850106,-0.05243778,-0.0066860667,-0.0030224898,0.032098126,0.033924438,-0.0139725655,-0.007993259,-0.00170748,0.030471953,-0.000023869736,0.06764873,0.0009757029,-0.008087076,0.01657444,0.0445571,0.033198915,0.07735573,-0.01850083,-0.048359837,0.0055070925,-0.015773864,-0.0040028836,0.015486157,-0.017174874,-0.021365391,-0.032998774,0.043506343,0.0076367515,0.016737057,0.020602342,-0.0445571,-0.08651233,-0.03227325,-0.033949457,0.00597618,0.02922105,0.0013275188,-0.00064225955,-0.03154773,0.02315418,0.017600179,-0.025230676,0.067048304,-0.039928764,0.00629516,0.011445746,-0.015986517,0.0032116887,0.028320402,0.014385363,-0.021553027,0.05101175,-0.025355767,0.025030533,0.0003373524,0.023216726,0.005638437,0.01786287,-0.01669953,0.006267015,-0.024517663,-0.06279524,-0.024217447,-0.043581396,-0.0020405324,0.009613176,-0.014698089,-0.011777234,-0.013146971,0.060293436,0.0066610486,-0.031422637,-2.5958641e-7,0.024267482,-0.005466438,-0.020427216,0.00328987,0.0147731425,-0.0139725655,-0.030421916,0.0045313896,-0.01708731,-0.023016581,-0.0058166906,-0.040304035,0.0016902802,-0.0015331358,0.0033836877,-0.053738713,0.032223213,-0.002925545,0.0038871753,0.06879956,-0.02784506,-0.0060043256,-0.0061982153,0.020990122,0.023266762,-0.0074366075,0.030046646,0.019976892,0.008055803,0.03304881,-0.031347584,0.010394989,-0.009907138,-0.0369266,-0.026394015,0.015361066,0.020990122,-0.04620829,0.07900692,0.027669935,0.019639147,-0.02329178,-0.023617014,-0.009681975,-0.03530043,-0.021615572,-0.025505874,-0.016849639,0.02329178,-0.02809524,-0.039428405,0.069149815,-0.009200378,0.020990122,-0.040304035,-0.018675955,0.039428405,0.077505834,-0.09551881,-0.01797545,0.016336769,-0.025343258,-0.0009131578,0.003586959,0.012746682,0.027294664,0.011014185,0.049335543,-0.059893146,-0.008318493,-0.0023423124,0.038377646,-0.0064609046,-0.031697836,0.052587885,-0.010764005,-0.004265573,0.046758685,-0.030722132,-0.010588879,0.017887887,-0.017112328,-0.019701693,-0.0155737195,0.014185219,-0.012471485,-0.07105119,-0.0019373331,0.0072302087,0.05198745,0.011045457,-0.017637707,0.019476531,0.068299204,0.04745919,0.059292715,0.030797187,-0.00052342395,0.013096935,-0.022916509,0.013359624,-0.016249206,0.053438496,0.0011015749,-0.051236913,-0.010445025,-0.06694823,0.0098821195,0.022428658,-0.0102824075,-0.01669953,-0.0112831285,0.019914346,-0.031597763,0.017500108,-0.0066860667,-0.0053851297,0.0011523927,0.0044688443,-0.026469069,-0.0013799004,0.012777955,-0.0054820743,-0.021027649,0.0031553982,-0.024342537,-0.03444982,0.0110266935,0.025793582,-0.00905027,0.04075436,-0.009681975,0.0049660774,-0.026118817,-0.0075992243,-0.071651615,0.01960162,0.059642967,0.015661282,-0.015898954,-0.019138787,0.026243906,-0.043231145,0.007061337,0.0025565294,-0.02784506,-0.09401773,-0.01367235,-0.009913391,-0.047108937,0.032298267,0.05659077,-0.034124583,-0.02937116,-0.033699278,-0.022578767,0.0059417807,-0.017387526,0.02200335,-0.042080317,-0.025718529,-0.021815715,-0.04658356,-0.019076243,0.020414706,0.035400502,0.00734279,-0.04408176,-0.037251838,0.014485436,-0.009056524,-0.024642752,0.027169574,0.0072114454,0.045132514,-0.019413985,-0.033298988,-0.018250648,0.063695885,0.013134462,-0.004350009,0.020051945,0.022503711,-0.024492646,0.0010015027,0.01045128,0.036426242,-0.07550439,0.0036088498,-0.07570454,0.0058385814,-0.033824366,-0.06349574,-0.028020186,-0.023066618,0.025893655,0.007799369,0.015423612,-0.041579954,-0.02090256,0.017450072,-0.0061169066,0.00029669813,-0.014648053,-0.004934805,0.037151765,-0.040679306,0.023429379,-0.00670483,0.009325468,0.01189607,-0.013509733,-0.0053601116,0.057691563,-0.031998053,0.015273503,-0.0006051234,0.0041811373,-0.009256668,-0.013272061,-0.014247764,0.0037745943,0.030321844,0.039303314,-0.053338427,0.01786287,0.05088666,0.006711085,-0.0016652622,0.020064455,-0.014185219,0.015498665,0.0042843367,0.015110886,-0.002181259,0.044156812,0.0033680515,0.0022000223,0.0064358865,0.024955478,0.016536914,0.0074115894,0.019801766,0.037877288,0.022853965,-0.122988604,0.0036463768,-0.041029558,0.002875509,0.033824366,-0.02494297,0.015898954,0.011921088,0.064946786,0.029546285,-0.016436841,0.010376225,-0.033749312,0.0129093,0.018625919,-0.0476093,0.0029443086,-0.017475089,-0.05013612,-0.019801766,-0.010651424,0.03557563,0.04530764,0.014097656,0.020965103,-0.0060981433,0.004065429,0.0067673754,-0.014397873,-0.0137724215,-0.0017684615,0.012208795,0.035375483,-0.045107495,-0.0148607055,0.017112328,0.013834967,0.0019576603,0.011996143,0.026619177,0.031647798,-0.019439004,0.0061763246,-0.01253403,0.023004072,-0.021678118,-0.017925413,0.03785227,0.0072114454,0.003299252,-0.007855659,0.0073052626,0.0075491886,0.010601387,0.00923165,-0.0067486116,0.008205912,-0.008062058,-0.00064734137,-0.00012714238,-0.013259552,0.0012532466,0.018163085,0.015498665,0.023504432,-0.008305984,-0.0627452,-0.0028473637,0.030572025,-0.012759192,0.009069033,-0.025618456,-0.017287454,0.00809333,0.023854686,0.015323539,0.014748124,-0.001452609,0.0052350215,-0.044106774,0.004709643,0.0034524873,-0.0022172222,0.001502645,-0.03254845,-0.003027181,-0.01847581,0.025168132,-0.021077685,-0.017575162,-0.0105388425,0.012265086,0.0025471475,0.008631218,-0.023066618,0.01644935,0.059592932,-0.013297079,0.03149769,0.018125558,-0.042980965,0.0014229001,-0.0048253513,-0.017687742,0.068299204,0.026769284,-0.024630243,-0.023829667,-0.027369717,-0.008399801,-0.007699297,-0.0088251075,-0.0072114454,0.013947548,0.005491456,0.025280712,-0.02252873,0.05834203,-0.009275432,-0.0035494321,-0.00068369566,0.008218421,-0.012802973,-0.018325701,-0.0043343725,0.012258831,0.0034556144,-0.00091237604,0.034099564,0.020915067,0.0011907015,0.00039696565,0.011514545,0.017675234,-0.0049129142,-0.016599458,-0.0015972444,-0.04570793,0.035750754,0.013322097,-0.01960162,-0.03887801,0.00956314,0.08015775,0.025718529,0.02999661,0.010976657,0.030146718,0.012884282,0.016311752,-0.0011500473,-0.015711319,0.0070988643,0.044532083,-0.0039372114,-0.028295385,-0.018813554,-0.024355046,-0.030972313,0.0060762526,0.014247764,0.019476531,-0.012527775,-0.0035775774,-0.01606157,0.05073655,0.020202054,-0.022691347,0.000043830405,0.0034368508,-0.041229703,0.006404614,-0.06419625,0.0021359138,0.022503711,0.008806344,-0.07290252,0.004118592,-0.009044016,-0.013059408,-0.036801513,0.03735191,-0.00091706694,0.004981714,-0.024204938,-0.012333886,0.014497944,-0.011583345,-0.0516372,-0.021540519,0.015798882,0.0013126644,0.00924416,0.017950432,0.013497223,0.019576604,-0.0065109404,-0.013784931,0.028245348,0.019226352,-0.02254124,-0.0062107244,0.040203962,-0.002381403,-0.030471953,-0.0075867157,0.050411317,-0.028320402,-0.005729127,-0.027820041,0.0051412038,-0.009100306,-0.011514545,-0.00617007,-0.008868889,0.005453929,-0.014510454,0.009513103,0.0057134912,0.0125715565,-0.014910742,-0.044732224,-0.037802234,-0.010420007,0.009388013,0.006592249,-0.004443826,-0.0133721335,-0.072452195,-0.016499387,-0.03274859,0.006326433,0.008568673,0.032873683,-0.00082090386,0.033499133,0.047759406,0.018901117,0.010307426,0.015135904,0.006173197,-0.024505153,0.00044993352,-0.009094051,-0.02684434,0.0030396897,-0.026569141,0.028395457,-0.032223213,0.006132543,-0.0054820743,0.006254506,-0.025493365,0.013559769,-0.013209516,-0.007542934,-0.020001909,-0.055139724,0.0135722775,-0.00551022,-0.022678837,0.008487364,0.03857779,-0.009075288,-0.03785227,0.009544376,-0.038402665,0.02329178,-0.02037718,-0.011677163,-0.016399315,0.05759149,-0.006611013,0.02139041,-0.0058166906,0.008599945,-0.013847476,0.026669214,-0.016511895,-0.018713482,-0.022291059,0.012246323,-0.02100263,-0.014923251,0.00037097037,-0.003085035,-0.014885724,0.025918672,0.0057197455,-0.028170295,0.024480136,-0.007530425,-0.005181858,-0.019038716,-0.048509948,0.016136626,0.025243185,-0.01973922,0.0036119772,-0.011470764,0.01569881,0.0058073085,-0.01621168,-0.022616293,-0.011633381,0.01632426,0.023617014,-0.001389282,-0.018713482,0.023867194,-0.0011993014,-0.024892934,-0.00021656226,0.012996863,0.031847943,-0.026544122,-0.027394736,-0.012746682,-0.01340966,0.0213779,0.022453675,-0.019188823,-0.01043877,-0.018638426,0.003977866,0.017187381,-0.015198449,-0.0020358416,-0.021290338,-0.019964382,-0.03592588,-0.022090914,-0.024367554,0.007780605,0.023829667,-0.0014135183,0.031147439,-0.017362509,0.03862783,0.016924692,0.03189798,0.006598504,-0.05909257,-0.04403172,-0.002888018,-0.0033680515,-0.014635543,0.013434678,-0.026343979,-0.0012985917,0.0007356862,0.004628334,0.02558093,0.027169574,0.0065359585,-0.005960544,0.013484715,-0.0106264055,0.009350486,-0.037301872,0.060593653,0.031722855,0.011508291,0.00011013794,0.042830855,0.022190986,0.0033586696,-0.053588606,-0.02455519,-0.0046439706,0.015110886,-0.007961986,-0.010764005,0.025893655,-0.022378622,0.0045720437,0.02594369,0.02847051,-0.015235976,-0.011458254,0.02036467,-0.023479415,0.005206876,-0.008299729,-0.026469069,-0.019989401,-0.009200378,0.0048441147,0.0067861388,-0.013747403,0.015748845,0.0006074689,-0.019251369,-0.0018450792,0.0098821195,-0.03757707,0.015611246,-0.007924459,0.0038652846,-0.01733749,-0.020026928,0.021703135,0.00028399366,-0.0036651404,-0.013634822,-0.022065897,-0.027419753,0.043206125,-0.021177758,0.028770726,0.00017737388,-0.013109445,0.02681932,-0.013997584,-0.011158038,0.025280712,-0.023054108,0.01810054,-0.033874404,-0.013759913,0.008618709,0.040579233,-0.04465717,0.046508506,-0.01911377,0.014135183,0.0043468815,-0.0359509,0.01644935,-0.011933597,-0.0005199058,0.004190519,-0.00759297,0.02696943,0.041429847,-0.012365158,-0.01669953,0.012959336,0.044857316,0.0015268812,0.036376204,-0.012946827,0.015748845,-0.019901838,0.0110829845,0.013021881,-0.01043877,-0.010463788,0.0014979541,0.0066547943,0.0359509,0.002905218,-0.046858758,0.04368147,0.0503863,0.04720901,0.041304756,0.031147439,0.027920114,0.005563383,0.049285505,0.017274946,-0.024792861,-0.010707714,0.00923165,-0.0105388425,0.020802487,-0.0060981433,0.008368528,0.0019388968,-0.026368996,0.013522241,0.043581396,-0.03177289,0.0017528252,-0.027920114,-0.00689872,-0.034099564,-0.032448377,-0.002855182,0.04745919,-0.032698557,-0.013284571,-0.01923886,0.014598017,0.0126278475,-0.048635036,0.003077217,-0.0015925536,0.0033711786,-0.0044188085,0.010101027,0.034524873,0.012077451,0.0031835434,-0.0033430334,-0.007255227,0.019589113,-0.004328118,0.045007423,-0.024605226,0.04090447,-0.015974008,-0.00087719446,0.006861193,0.027419753,0.028445492,-0.006442141,0.015723828,-0.0028645636,-0.0148607055,-0.015811391,0.028195312,0.005872981,0.015548701,0.02316669,0.008030785,-0.0258186,-0.01543612,0.01783785,0.009769538,0.043831576,0.012659119,-0.0141727105,-0.008074567,-0.015398594,-0.036025953,-0.0058667264,0.0025299476,-0.00009230283,0.002695692,-0.020189544,0.009419286,0.002695692,0.016511895,-0.010914112,-0.051061787,-0.015473647,-0.01506085,-0.0038215031,0.02784506,-0.016149133,0.0052475305,0.023967266,-0.004859751,-0.0036495042,-0.007392826,0.0023532577,-0.032573465,-0.0242925,0.0034087056,-0.003621359,-0.011602108,-0.0027598008,0.001540172,0.013484715,0.0022719493,-0.014160201,0.008675,0.0102824075,0.020477252,-0.014948268,-0.0031991797,-0.002121841,0.019889329,-0.0007548407,-0.022891492,0.018263157,-0.0015902082,0.0062388694,0.015073359,-0.0062982873,-0.016949711,-0.004265573,0.017112328,0.00057267817,-0.030346863,0.0018904244,-0.0100635,-0.004972332,-0.010107282,-0.0061294157,0.027995167,0.007693042,-0.012540285,-0.004697134,0.042205404,-0.015298521,0.0015761355,0.010857822,-0.017249927,-0.016399315,0.001469027,-0.03762711,-0.04152992,0.027244627,0.0004737788,-0.017637707,0.017700251,-0.016399315,0.00094443036,-0.02100263,0.009832083,0.030446934,-0.021765681,-0.00040615196,-0.019226352,-0.028145276,-0.03467498,0.012102469,0.0062763966,0.0033555424,0.014535472,-0.006107525,0.017562652,-0.010939131,-0.03820252,0.039328333,0.01100793,-0.012996863,0.061844554,0.022929018,-0.0012149378,-0.0062513785,-0.011039203,0.0025596565,0.016987238,-0.004537644,-0.008975216,0.019313915,0.001059357,0.006698576,0.017437562,-0.0131719895,0.008268457,-0.022941528,0.009975937,0.02518064,-0.015898954,-0.029971592,-0.008143366,-0.0027707461,0.024993006,0.009119069,-0.016787093,-0.025243185,-0.005973053,0.012171268,0.004772188,-0.0012845191,-0.0002779346,-0.029020907,-0.034249675,-0.017387526,0.02022707,0.007455371,-0.00024451208,-0.029896537,0.011039203,-0.014022602,0.027569862,0.028445492,-0.025505874,-0.017550142,-0.0046564797,0.038552772,-0.029846502,-0.011001675,-0.010401243,0.011477018,-0.0045845527,0.021352883,0.02505555,0.011495782,-0.013522241,-0.047784425,0.019476531,-0.004350009,0.009794557,0.004909787,-0.018075522,-0.0213779,-0.009006488,-0.023516942,0.011408218,-0.0427558,-0.048635036,0.025593437,0.00172468,0.00023591214,-0.010294916,0.0063639595,0.025843618,-0.033073828,0.0006019962,0.024054829,0.028045204,0.046658613,0.0037996122,-0.050561424,-0.005034877,0.014422891,0.008787581,0.0016042808,0.0012337012,0.015286013,0.0021937678,-0.0032210704,0.038052414,0.026168853,0.026168853,-0.01481067,-0.015661282,-0.005256912,-0.025480857,-0.0029396177,0.0010820295,0.004794079,-0.035000216,-0.026444051,0.008818854,0.006973774,0.012821737,0.006054362,-0.024655262,-0.0069049746,-0.0029396177,-0.007936968,0.03404953,0.010545096,-0.011433236,-0.00061763247,0.0059292717,-0.052487813,-0.008531146,-0.009656957,-0.02266633,-0.022828946,-0.002930236,0.015673792,-0.0046658614,-0.011739708,0.01619917,-0.015035832,0.02594369,-0.007868168,0.022065897,0.04110461,-0.014623035,-0.012602829,0.003349288,-0.016349278,0.0027598008,-0.00092723046,-0.030797187,-0.004503244,-0.0242925,0.008174639,0.021452954,-0.007555443,-0.00446259,0.035275415,-0.0029896537,0.011133021,0.027644916,-0.008637473,-0.023429379,-0.0021859498,0.011289383,-0.0007571861,-0.026394015,0.024880424,-0.0071926815,-0.03377433,0.040429126,0.03667642,0.011733453,-0.009388013,0.009075288,-0.015361066,-0.0035150324,-0.0095381215,-0.015085868,0.0043844087,0.023004072,-0.027669935,0.0172124,0.000673923,-0.0057134912,0.019313915,-0.002875509,0.016636986,-0.016286733,-0.0142978,-0.0034431054,-0.012665374,-0.030547006,-0.02075245,-0.0046596066,-0.019188823,-0.00007246431,-0.0013908457,-0.011014185,0.004834733,0.038127467,0.0057322546,-0.015498665,0.003977866,0.04668363,-0.012252577,0.023341816,-0.0011164293,0.0050036046,0.023504432,-0.0035025233,0.019026207,-0.013397152,0.0054414202,-0.029496249,-0.016549421,-0.016024044,-0.010576369,-0.009738266,-0.010244881,0.022378622,-0.0016433714,-0.011376946,0.028445492,-0.0184633,-0.00471277,-0.020990122,0.0068549383,-0.0024001666,0.022378622,0.015336048,0.01619917,-0.026143834,0.0498359,0.0054132747,0.02379214,-0.017012255,-0.049810883,-0.021615572,-0.0054101474,0.0063764686,0.005106804,0.022266041,0.010989167,-0.0052850572,-0.025318239,-0.009400522,0.0071864272,0.028195312,-0.02505555,0.022053387,0.0023876575,0.009688229,-0.03444982,0.019013697,-0.0045814253,0.009769538,-0.034599926,0.009957173,-0.023379343,-0.0041592466,-0.011014185,0.026644194,0.011301892,-0.023316797,0.012734174,-0.0140851475,0.013272061,0.00042530638,-0.03760209,-0.021077685,-0.019926855,0.010088518,-0.011539564,-0.0100259725,-0.042455584,-0.021815715,0.00029865265,-0.03960353,0.0044719717,0.0027738733,-0.006611013,0.0013986639,0.008518637,-0.011852289,0.036776494,-0.0030084173,0.0019858056,-0.013872494,-0.019789256,-0.014035111,-0.0049785865,-0.0005969144,-0.0023454397,0.028645637,-0.001619917,-0.036126025,0.010995422,0.023591995,-0.054088965,0.0063514505,-0.02468028,0.022253532,0.010457533,0.00019721239,-0.024455117,-0.033098843,0.036751475,-0.01607408,0.0026268924,-0.033899423,-0.007355299,-0.014185219,-0.015773864,-0.004728406,0.009488085,0.021415427,0.03847772,-0.044356953,0.031047367,0.009732011,0.038402665,0.011989888,0.011452001,0.0034399782,0.017575162,0.01975173,-0.02504304,0.00011160384,0.0059261443,-0.0026253287,-0.04430692,0.038652845,0.026644194,-0.0102824075,0.021690626,-0.015798882,-0.020139508,-0.012033669,0.004806588,-0.063295595,0.0036495042,0.010982912,-0.020602342,0.0027363463,0.033699278,0.0074866433,0.03820252,-0.015135904,-0.0066673034,-0.046858758,0.012458975,-0.0012243196,0.009169105,-0.0028567456,-0.0049410597,0.017037274,-0.023529451,0.017700251,-0.01708731,0.0244301,0.02962134,-0.013684859,0.035500575,0.043506343,-0.0036276134,0.022753892,0.0038934299,-0.0014002275,-0.005622801,0.025218168,-0.0040497924,0.017750287,-0.022416148,-0.003172598,0.010213608,0.000090836926,0.004884769,-0.013534751,-0.010663932,-0.0033461605,-0.01431031,-0.017074801,0.0045908075,0.03407455,-0.0054883286,0.009694484,0.018175595,-0.012515266,-0.0001456127,-0.0044719717,0.0029615085,0.008556164,0.017887887,-0.027744988,-0.0035588138,-0.010782768,0.021165248,-0.05634059,0.0041404827,0.019639147,0.022178477,-0.03645126,0.0048128422,-0.006132543]

## Single vector search

In [8]:
# IMPORTANT: Before you run this code, make sure the documents were successfully
# created in the previous step. Sometimes it may take a few seconds for the index to be ready.
# Check the "Document count" for the index in the Azure portal.

# Execute vector search
if vector:
    try:
        vector_query = VectorizedQuery(
            vector=vector,
            k_nearest_neighbors=5,
            fields="DescriptionVector",
            kind="vector",
            exhaustive=True
        )

        results = search_client.search(
            vector_queries=[vector_query],
            select=["HotelId", "HotelName", "Description", "Category", "Tags"],
            top=5,
            include_total_count=True
        )

        print(f"Total results: {results.get_count()}")
        for result in results:
            doc = result  # result is a dict-like object
            print(f"- HotelId: {doc['HotelId']}, HotelName: {doc['HotelName']}, Category: {doc.get('Category')}")
    except Exception as ex:
        print("Vector search failed:", ex)
else:
    print("No vector loaded, skipping search.")

Total results: 5
- HotelId: 48, HotelName: Nordick's Valley Motel, Category: Boutique
- HotelId: 13, HotelName: Luxury Lion Resort, Category: Luxury
- HotelId: 4, HotelName: Sublime Palace Hotel, Category: Boutique
- HotelId: 49, HotelName: Swirling Currents Hotel, Category: Suite
- HotelId: 2, HotelName: Old Century Hotel, Category: Boutique


## Single vector search with filter

In [9]:
if vector:
    try:
        vector_query = VectorizedQuery(
            vector=vector,
            k_nearest_neighbors=5,
            fields="DescriptionVector",
            kind="vector",
            exhaustive=True
        )

        results = search_client.search(
            vector_queries=[vector_query],
            filter="Tags/any(tag: tag eq 'free wifi')",  # <--- NOTICE THE FILTER IS ADDED HERE
            select=["HotelId", "HotelName", "Description", "Category", "Tags"],
            top=7,
            include_total_count=True
        )

        print(f"Total filtered results for 'free wifi': {results.get_count()}")
        for result in results:
            doc = result
            print(f"- HotelId: {doc['HotelId']}, HotelName: {doc['HotelName']}, Tags: {doc.get('Tags')}")
    except Exception as ex:
        print("Vector search with filter failed:", ex)
else:
    print("No vector loaded, skipping search.")

Total filtered results for 'free wifi': 2
- HotelId: 48, HotelName: Nordick's Valley Motel, Tags: ['continental breakfast', 'air conditioning', 'free wifi']
- HotelId: 2, HotelName: Old Century Hotel, Tags: ['pool', 'free wifi', 'air conditioning', 'concierge']


## Vector query with a geo filter

In [10]:
if vector:
    try:
        vector_query = VectorizedQuery(
            vector=vector,
            k_nearest_neighbors=5,
            fields="DescriptionVector",
            kind="vector",
            exhaustive=True
        )

        results = search_client.search(
            include_total_count=True,
            top=5,
            select=[
                "HotelId", "HotelName", "Category", "Description", "Address/City", "Address/StateProvince"
            ],
            facets=["Address/StateProvince"],
            filter="geo.distance(Location, geography'POINT(-77.03241 38.90166)') le 300",
            vector_filter_mode="postFilter",
            vector_queries=[vector_query]
        )

        print(f"Total filtered results based on geo coordinates: {results.get_count()}")
        for result in results:
            doc = result
            score = result.get("@search.score", "N/A")
            print(f"- HotelId: {doc['HotelId']}")
            print(f"  HotelName: {doc['HotelName']}")
            print(f"  Score: {score}")
            print(f"  City/State: {doc['Address']['City']}, {doc['Address']['StateProvince']}")
            # print(f"  Semantic Caption: {caption}\n")
            print(f"  Description: {doc.get('Description')}\n")

    except Exception as ex:
        print("Semantic hybrid search failed:", ex)
else:
    print("No vector loaded, skipping search.")

Total filtered results based on geo coordinates: 2
- HotelId: 48
  HotelName: Nordick's Valley Motel
  Score: 0.6605852246284485
  City/State: Washington D.C., null
  Description: Only 90 miles (about 2 hours) from the nation's capital and nearby most everything the historic valley has to offer. Hiking? Wine Tasting? Exploring the caverns? It's all nearby and we have specially priced packages to help make our B&B your home base for fun while visiting the valley.

- HotelId: 49
  HotelName: Swirling Currents Hotel
  Score: 0.602634072303772
  City/State: Arlington, VA
  Description: Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs.



## 

## Hybrid search

In [11]:
if vector:
    try:
        vector_query = VectorizedQuery(
            vector=vector,
            k_nearest_neighbors=5,
            fields="DescriptionVector",
            kind="vector",
            exhaustive=True
        )

        results = search_client.search(
            include_total_count=True,
            search_text="historic hotel walk to restaurants and shopping",  # keyword part
            select=["HotelId", "HotelName", "Description", "Category", "Tags"],
            top=5,
            vector_queries=[vector_query]
        )

        print(f"Total hybrid results: {results.get_count()}")
        for result in results:
            doc = result
            score = result.get("@search.score", "N/A")
            print(f"- Score: {score}")
            print(f"  HotelId: {doc['HotelId']}")            
            print(f"  HotelName: {doc['HotelName']}")
            print(f"  Description: {doc.get('Description')}")
            print(f"  Category: {doc.get('Category')}")
            print(f"  Tags: {doc.get('Tags', 'N/A')}\n")

    except Exception as ex:
        print("Hybrid search failed:", ex)
else:
    print("No vector loaded, skipping search.")    

Total hybrid results: 7
- Score: 0.03279569745063782
  HotelId: 4
  HotelName: Sublime Palace Hotel
  Description: Sublime Palace Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 19th century resort, updated for every modern convenience.
  Category: Boutique
  Tags: ['concierge', 'view', 'air conditioning']

- Score: 0.032522473484277725
  HotelId: 13
  HotelName: Luxury Lion Resort
  Description: Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.
  Category: Luxury
  Tags: ['bar', 'concierge', 'restaurant']

- Score: 0.03205128386616707
  HotelId: 48
  HotelName: Nordick's Valley Motel
  Description: Only

## Semantic hybrid search

In [12]:
if vector:
    try:
        vector_query = VectorizedQuery(
            vector=vector,
            k_nearest_neighbors=5,
            fields="DescriptionVector",
            kind="vector",
            exhaustive=True
        )

        results = search_client.search(
            include_total_count=True,
            search_text="historic hotel walk to restaurants and shopping",
            select=[
                "HotelId", "HotelName", "Category", "Description"
            ],
            query_type="semantic",
            semantic_configuration_name="my-semantic-config",
            top=5,            
            vector_queries=[vector_query]
        )

        print(f"Total semantic hybrid results: {results.get_count()}")
        for result in results:
            doc = result
            score = result.get("@search.score", "N/A")
            reranker_score = result.get("@search.reranker_score", "N/A")
            print(f"- Score: {score}")
            print(f"  Re-ranker Score: {reranker_score}")
            print(f"  HotelId: {doc['HotelId']}")
            print(f"  HotelName: {doc['HotelName']}")
            print(f"  Description: {doc.get('Description')}")
            print(f"  Category: {doc.get('Category')}")

    except Exception as ex:
        print("Semantic hybrid search failed:", ex)
else:
    print("No vector loaded, skipping search.")       

Total semantic hybrid results: 7
- Score: 0.0317460335791111
  Re-ranker Score: 2.6550590991973877
  HotelId: 49
  HotelName: Swirling Currents Hotel
  Description: Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs.
  Category: Suite
- Score: 0.03279569745063782
  Re-ranker Score: 2.599761724472046
  HotelId: 4
  HotelName: Sublime Palace Hotel
  Description: Sublime Palace Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 19th century resort, updated for every modern convenience.
  Category: Boutique
- Sc

## Clean up

In [13]:
index_client.delete_index(AZURE_SEARCH_INDEX)
print(f"Index '{AZURE_SEARCH_INDEX}' deleted successfully.")

Index 'hotels-sample-vector-index' deleted successfully.
